## CLIF Table One

Author: Kaveri Chhikara
Date v1: May 13, 2025

This script identifies the cohort of encounters with at least one ICU stay and then summarizes the cohort data into one table. 


#### Requirements

* Required table filenames should be `clif_patient`, `clif_hospitalization`, `clif_adt`, `clif_vitals`, `clif_labs`, `clif_medication_admin_continuous`, `clif_respiratory_support`, `clif_patient_assessments`
* Within each table, the following variables and categories are required.

| Table Name | Required Variables | Required Categories |
| --- | --- | --- |
| **clif_patient** | `patient_id`, `race_category`, `ethnicity_category`, `sex_category`, `death_dttm` | - |
| **clif_hospitalization** | `patient_id`, `hospitalization_id`, `admission_dttm`, `discharge_dttm`,`discharge_dttm`, `age_at_admission` | - |
| **clif_adt** |  `hospitalization_id`, `hospital_id`,`in_dttm`, `out_dttm`, `location_category` | - |
| **clif_vitals** | `hospitalization_id`, `recorded_dttm`, `vital_category`, `vital_value` | weight_kg |
| **clif_labs** | `hospitalization_id`, `lab_result_dttm`, `lab_order_dttm`, `lab_category`, `lab_value_numeric` | creatinine, bilirubin_total, po2_arterial, platelet_count |
| **clif_medication_admin_continuous** | `hospitalization_id`, `admin_dttm`, `med_name`, `med_category`, `med_dose`, `med_dose_unit` | norepinephrine, epinephrine, phenylephrine, vasopressin, dopamine, angiotensin(optional) |
| **clif_respiratory_support** | `hospitalization_id`, `recorded_dttm`, `device_category`, `mode_category`,  `fio2_set`, `lpm_set`, `resp_rate_set`, `peep_set`, `resp_rate_obs`, `tidal_volume_set`, `pressure_control_set`, `pressure_support_set` | - |
| **clif_patient_assessments** | `hospitalization_id`, `recorded_dttm` , `assessment_category`, `numerical_value`| `gcs_total` |
| **clif_crrt_therapy** | `hospitalization_id`, `recorded_dttm` | - |


# Cohort Identification


## Inclusion 
1. Adults
2. Patients with at least one ICU stay or those who had only emergency department or ward encounters and either died or received life support at any point. Life support is defined as the administration of any vasoactive drugs or respiratory support exceeding low-flow oxygen.

Respiratory support device: 'IMV', 'NIPPV', 'CPAP', 'High Flow NC'  

Vasoactive: 'norepinephrine', 'epinephrine', 'phenylephrine', 'vasopressin',
    'dopamine', 'angiotensin'

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import gc
from pathlib import Path
import json
from typing import Union
from tqdm import tqdm

import sys
import clifpy
import os

print("=== Environment Verification ===")
print(f"Python executable: {sys.executable}")
print(f"Python version: {sys.version}")
print(f"clifpy version: {clifpy.__version__}")
print(f"clifpy location: {clifpy.__file__}")

print("\n=== Python Path Check ===")
local_clifpy_path = "/Users/kavenchhikara/Desktop/CLIF/CLIFpy"
if any(local_clifpy_path in path for path in sys.path):
    print("⚠️  WARNING: Local CLIFpy still in path!")
    for path in sys.path:
        if local_clifpy_path in path:
            print(f"   Found: {path}")
else:
    print("✅ Clean environment - no local CLIFpy in path")

print(f"\n=== Working Directory ===")
print(f"Current directory: {os.getcwd()}")

=== Environment Verification ===
Python executable: /Users/kavenchhikara/Desktop/CLIF/CLIF-TableOne/.clif_table_one/bin/python
Python version: 3.9.6 (default, Apr 30 2025, 02:07:17) 
[Clang 17.0.0 (clang-1700.0.13.5)]
clifpy version: 0.2.0
clifpy location: /Users/kavenchhikara/Desktop/CLIF/CLIF-TableOne/.clif_table_one/lib/python3.9/site-packages/clifpy/__init__.py

=== Python Path Check ===
✅ Clean environment - no local CLIFpy in path

=== Working Directory ===
Current directory: /Users/kavenchhikara/Desktop/CLIF/CLIF-TableOne/code


In [2]:
# Load configuration
config_path = "../config/config.json"
with open(config_path, 'r') as f:
    config = json.load(f)

## import outlier json
# with open('../config/outlier_config.json', 'r', encoding='utf-8') as f:
#     outlier_cfg = json.load(f)

print(f"\n=� Configuration:")
print(f"   Data directory: {config['tables_path']}")
print(f"   File type: {config['file_type']}")
print(f"   Timezone: {config['timezone']}")


=� Configuration:
   Data directory: /Users/kavenchhikara/Library/CloudStorage/Box-Box/RCLIF_data/CLIF_2018_24/WIP_2_1
   File type: parquet
   Timezone: US/Central


## Required columns and categories

In [ ]:
print("\n" + "=" * 80)
print("Defining Required Data Elements")
print("=" * 80)

# Full patient table 

# Full hospitalization table 

# Full ADT table

# Vitals
vitals_required_columns = [
    'hospitalization_id',
    'recorded_dttm',
    'vital_category',
    'vital_value'
]
vitals_of_interest = ['heart_rate', 'respiratory_rate', 'sbp', 'dbp', 'map', 'spo2', 'weight_kg', 'height_cm']

#Labs
# labs_required_columns = [
#     'hospitalization_id',
#     'lab_result_dttm',
#     'lab_category',
#     'lab_value',
#     'lab_value_numeric'
# ]
# labs_of_interest = ['po2_arterial','pco2_arterial', 'ph_arterial','ph_venous', 'bicarbonate','so2_arterial',
#                     'sodium', 'potassium', 'chloride', 'calcium_total', 'magnesium', 'creatinine', 
#                     'bun', 'glucose_serum', 'lactate', 'hemoglobin' ]

# # Continuous administered meds
# meds_required_columns = [
#     'hospitalization_id',
#     'admin_dttm',
#     'med_name',
#     'med_category',
#     'med_dose',
#     'med_dose_unit'
# ]
# meds_of_interest = [
#     'norepinephrine', 'epinephrine', 'phenylephrine', 'vasopressin',
#     'dopamine', 'angiotensin', 'dobutamine', 'milrinone', 'isoproterenol',
#     'propofol', 'midazolam', 'lorazepam', 'dexmedetomidine', 
#     'vecuronium', 'rocuronium', 'cisatracurium', 'pancuronium'
# ]

# # Respiratory Support 
# rst_required_columns = [
#     'hospitalization_id',
#     'recorded_dttm',
#     'device_name',
#     'device_category',
#     'mode_name', 
#     'mode_category',
#     'tracheostomy',
#     'fio2_set',
#     'lpm_set',
#     'resp_rate_set',
#     'peep_set',
#     'resp_rate_obs',
#     'tidal_volume_set', 
#     'pressure_control_set',
#     'pressure_support_set',
#     'peak_inspiratory_pressure_set',
#     'peak_inspiratory_pressure_obs',
#     'plateau_pressure_obs',
#     'minute_vent_obs'
# ]

# # Full crrt table
# crrt_required_columns = [
#     'hospitalization_id',
#     'recorded_dttm',
#     'crrt_mode_category',
#     'blood_flow_rate',
#     'pre_filter_replacement_fluid_rate',
#     'post_filter_replacement_fluid_rate',
#     'dialysate_flow_rate',
#     'ultrafiltration_out'
# ]


Defining Required Data Elements


In [6]:
strobe_counts = {}

## Functions

In [ ]:
# def generate_hourly_sequence(group):
#     blk = group.name  # use group name from groupby
#     start_time = group['vent_episode_start_dttm'].iloc[0]
#     end_time   = group['vent_end_dttm_72h'].iloc[0]
#     hourly_timestamps = pd.date_range(start=start_time, end=end_time, freq='h')
#     return pd.DataFrame({
#         'hospitalization_id': blk,
#         'recorded_dttm': hourly_timestamps
#     })

# def calculate_ibw(height_cm, sex):
#     if pd.isna(height_cm) or pd.isna(sex):
#         return np.nan
#     height_inches = height_cm / 2.54
#     sex = str(sex).lower()
#     if sex == 'male':
#         return 50 + 2.3 * (height_inches - 60)
#     elif sex == 'female':
#         return 45.5 + 2.3 * (height_inches - 60)
#     else:
#         return np.nan

# def calculate_base_excess(ph, hco3):
#     """
#     Calculate Base Excess using simplified formula
#     BE = (HCO3 - 24.4) + (8.3 * (pH - 7.4))
#     """
#     return (hco3 - 24.4) + (8.3 * (ph - 7.4))

# def calculate_pf_ratio(po2, fio2):
#     """
#     Vectorized calculation of P/F ratio (PaO2/FiO2)
#     FiO2 should be as fraction (0.21-1.0), not percentage
#     Handles pandas Series input.
#     """
#     fio2 = fio2.copy()
#     # Convert percentage to fraction if needed
#     mask_pct = fio2 > 1
#     fio2[mask_pct] = fio2[mask_pct] / 100
#     # Set minimum fio2 to 0.21 (room air)
#     fio2 = fio2.clip(lower=0.21)
#     return po2 / fio2

# def process_crrt_waterfall(
#     crrt: pd.DataFrame,
#     *,
#     id_col: str = "hospitalization_id",
#     gap_thresh: Union[str, pd.Timedelta] = "2h",
#     infer_modes: bool = True,          # infer missing mode from numeric pattern
#     flag_missing_bfr: bool = True,     # add QC flag if blood-flow still NaN
#     wipe_unused: bool = True,          # null parameters not used by the mode
#     fix_islands: bool = True,          # relabel single-row SCUF islands
#     verbose: bool = True,
# ) -> pd.DataFrame:
#     """
#     Clean + episode-aware forward-fill for the CLIF `crrt_therapy` table.
#     Episode-aware clean-up and forward-fill of the CLIF `crrt_therapy` table.

#     The function mirrors the respiratory-support “waterfall” logic but adapts it to
#     the quirks of Continuous Renal Replacement Therapy (CRRT):

#     1. **Episode detection** - a new `crrt_episode_id` starts whenever  
#        • `crrt_mode_category` changes **OR**  
#        • the gap between successive rows exceeds *gap_thresh* (default 2 h).
#     2. **Numeric forward-fill inside an episode** - fills *only* the parameters
#        that are clinically relevant for the active mode.
#     3. **Mode-specific wiping** after filling, parameters that are **not used**
#        in the current mode (e.g. `dialysate_flow_rate` in SCUF) are nulled so
#        stale data never bleed across modes.
#     4. **Deduplication & ordering** guarantees exactly **one row per
#        `(id_col, recorded_dttm)`**, chronologically ordered.

#     Parameters
#     ----------
#     crrt : pd.DataFrame
#         Raw `crrt_therapy` table **in UTC**. Must contain the schema columns
#         defined on the CLIF website (see docstring footer).
#     id_col : str, default ``"hospitalization_id"``
#         Encounter-level identifier.
#     gap_thresh : str or pd.Timedelta, default ``"2h"``
#         Maximum tolerated gap **inside** an episode before a new episode is
#         forced. Accepts any pandas-parsable offset string (``"90min"``, ``"3h"``,
#         …) or a ``pd.Timedelta``.
#     verbose : bool, default ``True``
#         If *True* prints progress banners.

#     Returns
#     -------
#     pd.DataFrame
#         Processed CRRT DataFrame with

#         * ``crrt_episode_id`` (int32) - sequential per encounter,
#         * forward-filled numeric parameters **within** each episode,
#         * unused parameters blanked per mode,
#         * unique, ordered rows ``id_col, recorded_dttm``.

#     Add-ons v2.0
#     ------------
#     • Optional numeric-pattern inference of `crrt_mode_category`.
#     • Flags rows that *should* have blood-flow but don't.
#     • Optional fix for single-row modality islands (sandwiched rows).
#     • Optional wipe vs. keep of parameters not used by the active mode.

#     Key steps
#     ----------
#     0.  Lower-case strings, coerce numerics, **infer** mode when blank.
#     1.  **Relabel single-row SCUF islands** (if *fix_islands*).
#     2.  Detect `crrt_episode_id` (mode change or >gap_thresh).
#     3.  Forward-fill numeric parameters *within* an episode.
#     4.  QC flag → `blood_flow_missing_after_ffill` (optional).
#     5.  Wipe / flag parameters not valid for the mode (configurable).
#     6.  Deduplicate & order ⇒ one row per ``(id_col, recorded_dttm)``.
#     """
#     p = print if verbose else (lambda *_, **__: None)
#     gap_thresh = pd.Timedelta(gap_thresh)

#     # ───────────── Phase 0 — prep, numeric coercion, optional inference
#     p("✦ Phase 0: prep & numeric coercion (+optional mode inference)")
#     df = crrt.copy()

#     df["crrt_mode_category"] = df["crrt_mode_category"].str.lower()
#     # save original dialysate_flow_rate values
#     df["_orig_df"] = df["dialysate_flow_rate"]

#     # 0a) RAW SCUF DF‐OUT sanity check
#     # look for rows that are already labeled “scuf”
#     # and that have a non‐zero dialysate_flow_rate in the raw data
#     raw_scuf = df["crrt_mode_category"].str.lower() == "scuf"
#     raw_df_positive = df["_orig_df"].fillna(0) > 0

#     n_bad = (raw_scuf & raw_df_positive).sum()
#     if n_bad:
#         print(f"!!!  Found {n_bad} raw SCUF rows with dialysate_flow_rate > 0 (should be 0 or NA)")
#         print(" Converting these mode category to NA, keep recorded numerical values as the ground truth")
#         df.loc[raw_df_positive, "crrt_mode_category"] = np.nan
#     else:
#         print("!!! No raw SCUF rows had dialysate_flow_rate > 0")

#     NUM_COLS = [
#         "blood_flow_rate",
#         "pre_filter_replacement_fluid_rate",
#         "post_filter_replacement_fluid_rate",
#         "dialysate_flow_rate",
#         "ultrafiltration_out",
#     ]
#     NUM_COLS = [c for c in NUM_COLS if c in df.columns]
#     df[NUM_COLS] = df[NUM_COLS].apply(pd.to_numeric, errors="coerce")

#     #  any row whose original ultrafiltration_out was >0 must never be SCUF
#     def drop_scuf_on_positive_df(df, p):
#         bad_df  = df["_orig_df"].fillna(0) > 0
#         scuf_now = df["crrt_mode_category"] == "scuf"
#         n = (bad_df & scuf_now).sum()
#         if n:
#             p(f"→ Removing {n:,} SCUF labels on rows with DF>0")
#             df.loc[bad_df & scuf_now, "crrt_mode_category"] = np.nan
            

#     if infer_modes:
#         miss = df["crrt_mode_category"].isna()
#         pre  = df["pre_filter_replacement_fluid_rate"].notna()
#         post = df["post_filter_replacement_fluid_rate"].notna()
#         dial = df["dialysate_flow_rate"].notna()
#         bf   = df["blood_flow_rate"].notna()
#         uf   = df["ultrafiltration_out"].notna()
#         all_num_present = df[NUM_COLS].notna().all(axis=1)

#         df.loc[miss & all_num_present,                       "crrt_mode_category"] = "cvvhdf"
#         df.loc[miss & (~dial) & pre & post,                  "crrt_mode_category"] = "cvvh"
#         df.loc[miss & dial & (~pre) & (~post),               "crrt_mode_category"] = "cvvhd"
#         df.loc[miss & (~dial) & (~pre) & (~post) & bf & uf,  "crrt_mode_category"] = "scuf"

#         filled = (miss & df["crrt_mode_category"].notna()).sum()
#         p(f"  • numeric-pattern inference filled {filled:,} missing modes")
#         drop_scuf_on_positive_df(df, p)

#     # ───────────── Phase 1 — sort and *fix islands before episodes*
#     p("✦ Phase 1: sort + SCUF-island fix")
#     df = df.sort_values([id_col, "recorded_dttm"]).reset_index(drop=True)

#     if fix_islands:
#         # after sorting, BEFORE episode detection
#         prev_mode = df.groupby(id_col)["crrt_mode_category"].shift()
#         next_mode = df.groupby(id_col)["crrt_mode_category"].shift(-1)

#         scuf_island = (
#             (df["crrt_mode_category"] == "scuf") &
#             (prev_mode.notna()) & (next_mode.notna()) &     # ensure we have neighbours
#             (prev_mode == next_mode)                        # both neighbours agree
#         )

#         df.loc[scuf_island, "crrt_mode_category"] = prev_mode[scuf_island]
#         n_fixed = scuf_island.sum()
#         p(f"  • relabelled {n_fixed:,} SCUF-island rows")
#         drop_scuf_on_positive_df(df, p)


#     # ───────────── Phase 2 — episode detection (now with fixed modes)
#     p("✦ Phase 2: derive `crrt_episode_id`")
#     mode_change = (
#         df.groupby(id_col)["crrt_mode_category"]
#           .apply(lambda s: s != s.shift())
#           .reset_index(level=0, drop=True)
#     )
#     time_gap = df.groupby(id_col)["recorded_dttm"].diff().gt(gap_thresh).fillna(False)
#     df["crrt_episode_id"] = ((mode_change | time_gap)
#                               .groupby(df[id_col]).cumsum()
#                               .astype("int32"))

#     # ───────────── Phase 3 — forward-fill numerics inside episodes
#     p("✦ Phase 3: forward-fill numeric vars inside episodes")
#     tqdm.pandas(disable=not verbose, desc="ffill per episode")
#     df[NUM_COLS] = (
#         df.groupby([id_col, "crrt_episode_id"], sort=False, group_keys=False)[NUM_COLS]
#           .progress_apply(lambda g: g.ffill())
#     )

#     # QC: blood-flow still missing?
#     if flag_missing_bfr and "blood_flow_rate" in NUM_COLS:
#         need_bfr = df["crrt_mode_category"].isin(["scuf", "cvvh", "cvvhd", "cvvhdf"])
#         df["blood_flow_missing_after_ffill"] = need_bfr & df["blood_flow_rate"].isna()
#         p(f"  • blood-flow still missing where required: "
#           f"{df['blood_flow_missing_after_ffill'].mean():.1%}")
        
#     # Bridge tiny episodes
    
#     single_row_ep = (
#         df.groupby([id_col, "crrt_episode_id"]).size() == 1
#     ).reset_index(name="n").query("n == 1")
#     print("Bridging single row episodes")

#     rows_to_bridge = df.merge(single_row_ep[[id_col, "crrt_episode_id"]],
#                             on=[id_col, "crrt_episode_id"]).index
    
#     CAT_COLS = [c for c in ["crrt_mode_category"] if c in df.columns]

#     # Combine with the numeric columns we already had
#     BRIDGE_COLS = NUM_COLS + CAT_COLS

#     # Forward-fill (and back-fill just in case the island is the first row of the encounter)
#     df.loc[rows_to_bridge, BRIDGE_COLS] = (
#         df.loc[rows_to_bridge, BRIDGE_COLS]
#         .groupby(df.loc[rows_to_bridge, id_col])      # keep encounter boundaries
#         .apply(lambda g: g.ffill())          
#         .reset_index(level=0, drop=True)
#     )
#     drop_scuf_on_positive_df(df, p)
#     # ───────────── Phase 4 — wipe / flag unused parameters
#     p("✦ Phase 4: handle parameters not valid for the mode")
#     MODE_PARAM_MAP = {
#         "scuf":   {"blood_flow_rate", "ultrafiltration_out"},
#         "cvvh":   {"blood_flow_rate", "pre_filter_replacement_fluid_rate",
#                    "post_filter_replacement_fluid_rate", "ultrafiltration_out"},
#         "cvvhd":  {"blood_flow_rate", "dialysate_flow_rate", "ultrafiltration_out"},
#         "cvvhdf": {"blood_flow_rate", "pre_filter_replacement_fluid_rate","post_filter_replacement_fluid_rate",
#                    "dialysate_flow_rate", "ultrafiltration_out"},
#     }

#     wiped_totals = {c: 0 for c in NUM_COLS}
#     for mode, keep in MODE_PARAM_MAP.items():
#         mask = df["crrt_mode_category"] == mode
#         drop_cols = list(set(NUM_COLS) - keep)
#         if wipe_unused:
#             for col in drop_cols:
#                 wiped_totals[col] += df.loc[mask, col].notna().sum()
#             df.loc[mask, drop_cols] = np.nan
#         else:
#             for col in drop_cols:
#                 df.loc[mask & df[col].notna(), f"{col}_unexpected"] = True

#     if verbose and wipe_unused:
#         p("  • cells set → NA by wipe:")
#         for col, n in wiped_totals.items():
#             p(f"    {col:<35} {n:>8,}")
#     # ───────────── Phase 4a — SCUF‐specific sanity check
#     if "dialysate_flow_rate" in df.columns:
#         # only consider rows that were originally SCUF mode
#         # and whose original _orig_df was non‐zero/non‐NA
#         scuf_rows = df["crrt_mode_category"] == "scuf"
#         orig_bad = df["_orig_df"].fillna(0) > 0

#         # these are rows where the *original* data had UF>0 despite SCUF
#         bad_orig_scuf = scuf_rows & orig_bad

#         n_bad_orig = bad_orig_scuf.sum()
#         if n_bad_orig:
#             p(f"!!! {n_bad_orig} rows originally labeled SCUF had DF>0 (raw data); forcing DF→NA for those")
#             df.loc[bad_orig_scuf, "dialysate_flow_rate"] = np.nan
#         else:
#             p("!!! No SCUF rows with DF>0")

#     # then drop the helper column
#     df = df.drop(columns="_orig_df")

#     # ───────────── Phase 5 — deduplicate & order
#     p("✦ Phase 5: deduplicate & order")
#     pre = len(df)
#     df = (
#         df.drop_duplicates(subset=[id_col, "recorded_dttm"])
#           .sort_values([id_col, "recorded_dttm"])
#           .reset_index(drop=True)
#     )
#     p(f"  • dropped {pre - len(df):,} duplicate rows")

#     if verbose:
#         sparse = df[NUM_COLS].isna().all(axis=1).mean()
#         p(f"  • rows with all NUM_COLS missing: {sparse:.1%}")

#     p("[OK] CRRT waterfall complete.")
#     return df

## Cohort identification

In [3]:
print("\n" + "=" * 80)
print("Loading CLIF Tables")
print("=" * 80)

from clifpy.clif_orchestrator import ClifOrchestrator

# Initialize ClifOrchestrator
clif = ClifOrchestrator(
    data_directory=config['tables_path'],
    filetype=config['file_type'],
    timezone=config['timezone']
)



Loading CLIF Tables
ClifOrchestrator initialized.


## Step0: Load Core Tables

In [4]:
# ============================================================================
# STEP 0: Load Core Tables (Patient, Hospitalization, ADT)
# ============================================================================
print("\n" + "=" * 80)
print("Step 0: Load Core Tables (Patient, Hospitalization, ADT)")
print("=" * 80)
core_tables = ['patient', 'hospitalization', 'adt']

print(f"\nLoading {len(core_tables)} core tables...")
for table_name in core_tables:
    print(f"   Loading {table_name}...", end=" ")
    try:
        clif.load_table(table_name)
        table = getattr(clif, table_name)
        print(f"✓ ({len(table.df):,} rows)")
    except Exception as e:
        print(f"✗ Error: {e}")
        raise

print("\nCore tables loaded successfully!")


Step 0: Load Core Tables (Patient, Hospitalization, ADT)

Loading 3 core tables...
   Loading patient... ✓ (97,254 rows)
   Loading hospitalization... ✓ (166,814 rows)
   Loading adt... 

in_dttm: Naive datetime localized to US/Central. Please verify this is correct.
out_dttm: Naive datetime localized to US/Central. Please verify this is correct.


✓ (425,353 rows)

Core tables loaded successfully!


In [5]:
hosp_df = clif.hospitalization.df
adt_df = clif.adt.df

# Merge to get age information
all_encounters = pd.merge(
    hosp_df[["patient_id", "hospitalization_id", "admission_dttm", "discharge_dttm", 
             "age_at_admission", "discharge_category"]],
    adt_df[["hospitalization_id", "hospital_id", "in_dttm", "out_dttm", 
            "location_category", "location_type"]],
    on='hospitalization_id',
    how='inner'
)

In [6]:
# Check for duplicates by ['hospitalization_id', 'in_dttm', 'out_dttm']
dup_counts = all_encounters.duplicated(subset=['hospitalization_id', 'in_dttm', 'out_dttm']).sum()
if dup_counts > 0:
    print(f"Warning: {dup_counts} duplicate (hospitalization_id, in_dttm, out_dttm) entries found in all_encounters.")
else:
    print("No duplicate (hospitalization_id, in_dttm, out_dttm) entries found in all_encounters.")

No duplicate (hospitalization_id, in_dttm, out_dttm) entries found in all_encounters.


## Step1: Date & Age filter

In [7]:
all_encounters.columns

Index(['patient_id', 'hospitalization_id', 'admission_dttm', 'discharge_dttm',
       'age_at_admission', 'discharge_category', 'hospital_id', 'in_dttm',
       'out_dttm', 'location_category', 'location_type'],
      dtype='object')

In [ ]:
# ============================================================================
# STEP 1: Identify Adult Patients (Age >= 18) and Admissions 2018-2024
# ============================================================================
print("\n" + "=" * 80)
print("Step 1: Identifying Adult Patients (Age >= 18) and Admissions 2018-2024")
print("=" * 80)

print("Applying initial cohort filters...")

# Use only the relevant columns from all_encounters
adult_encounters = all_encounters[
    [
        'patient_id', 'hospitalization_id', 'admission_dttm', 'discharge_dttm',
        'age_at_admission', 'discharge_category', 'hospital_id',
        'in_dttm', 'out_dttm', 'location_category', 'location_type'
    ]
].copy()

if config['timezone'].lower() == "mimic":
    # MIMIC: only age >= 18, no admit year restriction
    adult_encounters = adult_encounters[
        (adult_encounters['age_at_admission'] >= 18) & (adult_encounters['age_at_admission'].notna())
    ]
else:
    # Other sites: age >= 18 and admission between 2018-2024 inclusive
    adult_encounters = adult_encounters[
        (adult_encounters['age_at_admission'] >= 18) &
        (adult_encounters['age_at_admission'].notna()) &
        (adult_encounters['admission_dttm'].dt.year >= 2018) &
        (adult_encounters['admission_dttm'].dt.year <= 2024)
    ]


print(f"\nFiltering Results:")
print(f"   Total hospitalizations: {len(all_encounters['hospitalization_id'].unique()):,}")
print(f"   Adult hospitalizations (age >= 18, 2018-2024): {len(adult_encounters['hospitalization_id'].unique()):,}")
print(f"   Excluded (age < 18 or outside 2018-2024): {len(all_encounters['hospitalization_id'].unique()) - len(adult_encounters['hospitalization_id'].unique()):,}")


strobe_counts["0_total_hospitalizations"] = len(all_encounters['hospitalization_id'].unique())
strobe_counts["1_adult_hospitalizations"] = len(adult_encounters['hospitalization_id'].unique())
# Get list of adult hospitalization IDs for filtering
adult_hosp_ids = set(adult_encounters['hospitalization_id'].unique())
print(f"\n   Unique adult hospitalization IDs: {len(adult_hosp_ids):,}")


Step 1: Identifying Adult Patients (Age >= 18) and Admissions 2018-2024
Applying initial cohort filters...

Filtering Results:
   Total hospitalizations: 166,781
   Adult hospitalizations (age >= 18, 2018-2024): 166,781
   Excluded (age < 18 or outside 2018-2024): 0

   Unique adult hospitalization IDs: 166,781


### Stitch hospitalizations 

If the `id_col` supplied by user is `hospitalization_id`, then we combine multiple `hospitalization_ids` into a single `encounter_block` for patients who transfer between hospital campuses or return soon after discharge. Hospitalizations that have a gap of **6 hours or less** between the discharge dttm and admission dttm are put in one encounter block.

If the `id_col` supplied by user is `hospitalization_joined_id` from the hospitalization table, then we consider the user has already stitched similar encounters, and we will consider that as the primary id column for all table joins moving forward.

In [14]:
from clifpy.utils.stitching_encounters import stitch_encounters

# Instead of multiple copies, work with references and clean up
hosp_filtered = clif.hospitalization.df[clif.hospitalization.df['hospitalization_id'].isin(adult_hosp_ids)]
adt_filtered = clif.adt.df[clif.adt.df['hospitalization_id'].isin(adult_hosp_ids)]

hosp_stitched, adt_stitched, encounter_mapping = stitch_encounters(
    hospitalization=hosp_filtered,
    adt=adt_filtered,
    time_interval=6  
)

# Direct assignment without additional copies
clif.hospitalization.df = hosp_stitched
clif.adt.df = adt_stitched

# Store the encounter mapping in the orchestrator for later use
clif.encounter_mapping = encounter_mapping

# Clean up intermediate variables
del hosp_filtered, adt_filtered
gc.collect()

0

In [15]:
# After your stitching code, add these calculations:

# Calculate stitching statistics
strobe_counts['1b_before_stitching'] = len(adult_hosp_ids)  # Original adult hospitalizations
strobe_counts['1b_after_stitching'] = len(hosp_stitched['encounter_block'].unique())  # Unique encounter blocks after stitching
strobe_counts['1b_stitched_hosp_ids'] = strobe_counts['1b_before_stitching'] - strobe_counts['1b_after_stitching']  # Number of hospitalizations that were linked

print(f"\nEncounter Stitching Results:")
print(f"   Number of unique hospitalizations before stitching: {strobe_counts['1b_before_stitching']:,}")
print(f"   Number of unique encounter blocks after stitching: {strobe_counts['1b_after_stitching']:,}")
print(f"   Number of linked hospitalization ids: {strobe_counts['1b_stitched_hosp_ids']:,}")

# Optional: Show the encounter mapping details
print(f"\nEncounter Mapping Details:")
print(f"   Total encounter mappings created: {len(encounter_mapping):,}")
if len(encounter_mapping) > 0:
    # Show some examples of how many original hospitalizations were combined
    mapping_counts = encounter_mapping.groupby('encounter_block').size()
    print(f"   Encounter blocks with multiple hospitalizations: {(mapping_counts > 1).sum():,}")
    print(f"   Maximum hospitalizations combined into one block: {mapping_counts.max()}")


Encounter Stitching Results:
   Number of unique hospitalizations before stitching: 166,781
   Number of unique encounter blocks after stitching: 166,644
   Number of linked hospitalization ids: 137

Encounter Mapping Details:
   Total encounter mappings created: 166,781
   Encounter blocks with multiple hospitalizations: 136
   Maximum hospitalizations combined into one block: 3


In [16]:
cohort_df = encounter_mapping.copy()

## Step2: Identify CRRT encounters 

In [ ]:
print(f"\nLoading crrt_therapy table...")
try:
    
    
    print(f"   CRRT therapy loaded: {len(clif.crrt_therapy.df):,} rows")
    print(f"   Unique CRRT therapy hospitalizations: {clif.crrt_therapy.df['hospitalization_id'].nunique()}")
except Exception as e:
    print(f"   CRRT therapy not available or error: {e}")


Loading crrt_therapy table...
   CRRT therapy loaded: 403,322 rows
   Unique CRRT therapy hospitalizations: 3152


In [18]:
# Update CRRT therapy DataFrame with encounter blocks
clif.crrt_therapy.df = clif.crrt_therapy.df.merge(
    clif.encounter_mapping[['hospitalization_id', 'encounter_block']],
    on='hospitalization_id',
    how='left'
)

n_crrt_hosp = clif.crrt_therapy.df['hospitalization_id'].nunique()
n_crrt_blocks = clif.crrt_therapy.df['encounter_block'].nunique()
crrt_hosp_ids = set(clif.crrt_therapy.df['hospitalization_id'].unique())

print(f"Updated CRRT therapy DataFrame:")
print(f"   Total CRRT records: {len(clif.crrt_therapy.df):,}")
print(f"   Records with encounter blocks: {clif.crrt_therapy.df['encounter_block'].notna().sum():,}")
print(f"   Unique encounter blocks in CRRT data: {n_crrt_blocks}")
print(f"   Unique hospitalizations  in CRRT data: {n_crrt_hosp}")

strobe_counts["2_crrt_hospitalizations"] = n_crrt_hosp
strobe_counts["2_crrt_blocks"] = n_crrt_blocks

# Filter cohort_df to only hospitalizations present in CRRT data
cohort_df = cohort_df[cohort_df['hospitalization_id'].isin(crrt_hosp_ids)].copy()

Updated CRRT therapy DataFrame:
   Total CRRT records: 403,322
   Records with encounter blocks: 403,322
   Unique encounter blocks in CRRT data: 3152
   Unique hospitalizations  in CRRT data: 3152


## Step3: Exclude ESRD encounters

Prior to admission ICD codes for ESRD

In [19]:
print(f"\nLoading Hospital dx table...")
try:
    clif.load_table(
        'hospital_diagnosis',
        filters={'hospitalization_id': list(crrt_hosp_ids)}
    )
    print(f"   Hospital dx loaded: {len(clif.hospital_diagnosis.df):,} rows")
    print(f"   Unique Hospital dx hospitalizations: {clif.hospital_diagnosis.df['hospitalization_id'].nunique()}")

    print("Merge encounter blocks with diagnosis")
    clif.hospital_diagnosis.df = clif.hospital_diagnosis.df.merge(
                    clif.encounter_mapping[['hospitalization_id', 'encounter_block']],
                    on='hospitalization_id',
                    how='left')

    n_dx_hosp = clif.hospital_diagnosis.df['hospitalization_id'].nunique()
    n_dx_blocks = clif.hospital_diagnosis.df['encounter_block'].nunique()
    cohort_hosp_ids = set(clif.hospital_diagnosis.df['hospitalization_id'].unique())
    cohort_blocks = set(clif.hospital_diagnosis.df['encounter_block'].unique())
    print(f"   Total Hospital dx records: {len(clif.hospital_diagnosis.df):,}")
    print(f"   Records with encounter blocks: {clif.hospital_diagnosis.df['encounter_block'].notna().sum():,}")
    print(f"   Unique encounter blocks in Hospital dx data: {n_dx_blocks}")
    print(f"   Unique hospitalizations  in Hospital dx data: {n_dx_hosp}")
except Exception as e:
    print(f"   Hospital dx not available or error: {e}")


Loading Hospital dx table...
   Hospital dx loaded: 108,880 rows
   Unique Hospital dx hospitalizations: 3124
Merge encounter blocks with diagnosis
   Total Hospital dx records: 108,880
   Records with encounter blocks: 108,880
   Unique encounter blocks in Hospital dx data: 3124
   Unique hospitalizations  in Hospital dx data: 3124


In [20]:
hospital_diagnosis_df = clif.hospital_diagnosis.df.copy()

print("Hospital dx column names :", hospital_diagnosis_df.columns)
# Clean and standardize diagnosis codes
hospital_diagnosis_df['diagnosis_code'] = hospital_diagnosis_df['diagnosis_code'].str.replace('.', '').str.lower()

if 'present_on_admission' in hospital_diagnosis_df.columns:
    hospital_diagnosis_df = hospital_diagnosis_df.rename(columns={'present_on_admission': 'poa_present'})

# Check present_on_admission column type and standardize to int8
if 'poa_present' in hospital_diagnosis_df.columns:
    # Only allow 1 (present on admission) or 0 (not present on admission)
    # Any other value (including Exempt, Unknown, Unspecified, NA) is set to 0
    hospital_diagnosis_df['poa_present'] = hospital_diagnosis_df['poa_present'].astype(str).str.lower()
    hospital_diagnosis_df['poa_present'] = hospital_diagnosis_df['poa_present'].map(
        {'yes': 1, 'y': 1, 'true': 1, '1': 1, 'no': 0, 'n': 0, 'false': 0, '0': 0}
    ).fillna(0).astype('int8')

Hospital dx column names : Index(['hospitalization_id', 'diagnosis_code', 'diagnosis_code_format',
       'diagnosis_primary', 'poa_present', 'encounter_block'],
      dtype='object')


In [21]:
# Define ESRD diagnosis codes
# Let's debug why we're not finding ESRD codes
esrd_codes = [
    'z992',    # Dependence on renal dialysis
    'z9115',   # Patient's noncompliance with renal dialysis
    'i120',    # Hypertensive chronic kidney disease with stage 5 CKD or ESRD
    'n186',    # End stage renal disease
    'i132',    # Hypertensive heart and chronic kidney disease with heart failure and ESRD
    'z992',    # Dependence on renal dialysis (alternate code)
    'i120',    # Hypertensive chronic kidney disease with stage 5 CKD or ESRD (alternate code)
    'z91158',  # Patient's noncompliance with renal dialysis (alternate code)
    'i1311',   # Hypertensive heart and chronic kidney disease with heart failure and stage 5 CKD
    'i132',    # Hypertensive heart and chronic kidney disease with ESRD (alternate code)
    '5856',     #ICD9 :End stage renal disease
    '40391',    #ICD9: Hypertensive chronic kidney disease, unspecified, with chronic kidney disease stage V or end stage renal disease
    '40311',     #ICD9: Hypertensive chronic kidney disease, benign, with chronic kidney disease stage V or end stage renal disease
    'v4511',     #ICD9: Renal dialysis status
    'v4512'     #ICD9: Noncompliance with renal dialysis
]

# Get hospitalization IDs with ESRD diagnoses and print debug info
print("\nNumber of rows matching ESRD codes:", hospital_diagnosis_df['diagnosis_code'].isin(esrd_codes).sum())


# Count how many ESRD codes have present_on_admission = 1, 0, or NA
esrd_poa_counts = hospital_diagnosis_df[
    hospital_diagnosis_df['diagnosis_code'].isin(esrd_codes)
]['poa_present'].value_counts(dropna=False)
print("Present_on_admission values for ESRD codes:")
print(esrd_poa_counts)

# Use a more inclusive approach for ESRD identification
# Include cases where present_on_admission is 1 OR NA (assuming NA means unknown/possible)
esrd_mask = (
    hospital_diagnosis_df['diagnosis_code'].isin(esrd_codes) & 
    ((hospital_diagnosis_df['poa_present'] == 1) | 
        (hospital_diagnosis_df['poa_present'].isna()))
)
hosp_ids_with_esrd = hospital_diagnosis_df[esrd_mask]['hospitalization_id'].unique()
blocks_with_esrd = hospital_diagnosis_df[esrd_mask]['encounter_block'].unique()

print(f"Hospitalizations with ESRD (including NA present_on_admission): {len(hosp_ids_with_esrd)}")


strobe_counts['3_hospitalizations_with_esrd'] = len(hosp_ids_with_esrd)
strobe_counts['3_encounter_blocks_with_esrd'] = len(blocks_with_esrd)


# Filter out hospitalizations with ESRD
cohort_df = cohort_df[~cohort_df['hospitalization_id'].isin(hosp_ids_with_esrd)].copy()
cohort_hosp_ids = set(cohort_df['hospitalization_id'].unique())
cohort_blocks = set(cohort_df['encounter_block'].unique())
# Create cohort subset excluding hospitalizations with ESRD
strobe_counts['3_encounter_blocks_without_esrd'] = len(cohort_blocks)  # Count blocks without ESRD
strobe_counts['3_hospitalizations_without_esrd'] = len(cohort_hosp_ids)  # Count hospitalizations without ESRD

strobe_counts


Number of rows matching ESRD codes: 1680
Present_on_admission values for ESRD codes:
poa_present
1    1588
0      92
Name: count, dtype: int64
Hospitalizations with ESRD (including NA present_on_admission): 905


{'0_total_hospitalizations': 166781,
 '1_adult_hospitalizations': 166781,
 '1b_before_stitching': 166781,
 '1b_after_stitching': 166644,
 '1b_stitched_hosp_ids': 137,
 '2_crrt_hospitalizations': 3152,
 '2_crrt_blocks': 3152,
 '3_hospitalizations_with_esrd': 905,
 '3_encounter_blocks_with_esrd': 905,
 '3_encounter_blocks_without_esrd': 2247,
 '3_hospitalizations_without_esrd': 2247}

## Cohort Sanity Check

In [22]:
crrt_df = clif.crrt_therapy.df

### 1.  AKI Codes 

In [23]:
# AKI Codes Sanity check

# Define AKI ICD-10 codes
aki_codes = [
    # ICD-10 codes for acute kidney injury
    'n170', 'n171', 'n172', 'n178', 'n179',  # Acute kidney failure codes
    'r34',   # Anuria and oliguria
    'n990', # Post-procedural kidney failure
    't795',  # Traumatic anuria
    '5845',  # ICD9 Acute kidney failure with lesion of tubular necrosis
    '5849',  # ICD9- Acute kidney failure, unspecified
    "5848"    # ICD9 - Acute kidney failure with other specified pathological lesion in kidney
]

# Filter to non-ESRD encounters first
non_esrd_encounters = hospital_diagnosis_df[hospital_diagnosis_df['encounter_block'].isin(cohort_df['encounter_block'])]

# Create mask for AKI diagnoses on the filtered data
aki_mask = non_esrd_encounters['diagnosis_code'].isin(aki_codes)

# Get encounter blocks with AKI diagnoses
blocks_with_aki = non_esrd_encounters[aki_mask]['encounter_block'].unique()
total_non_esrd_blocks = cohort_df['encounter_block'].nunique()
strobe_counts['3_encounter_blocks_with_AKI_no_esrd'] = len(blocks_with_aki) 

# Calculate percentage
aki_percentage = (len(blocks_with_aki) / total_non_esrd_blocks) * 100

print(f"\nPercentage of non-ESRD encounter blocks with AKI codes: {aki_percentage:.1f}%")
print(f"({len(blocks_with_aki)} out of {total_non_esrd_blocks} blocks)")
strobe_counts['3_Percentage_non_ESRD_encounter_blocks_with_AKI_codes'] = aki_percentage
# Show sample of AKI diagnoses
aki_diagnoses = non_esrd_encounters[aki_mask][['hospitalization_id', 'diagnosis_code','poa_present']].drop_duplicates()
print("\nSample of AKI-related diagnoses found: ")
aki_diagnoses['diagnosis_code'].value_counts()


Percentage of non-ESRD encounter blocks with AKI codes: 97.9%
(2199 out of 2247 blocks)

Sample of AKI-related diagnoses found: 


diagnosis_code
n170    1385
n179     897
r34      286
n990       9
n178       5
n171       1
n172       1
Name: count, dtype: int64

### 2. ICU Location

In [24]:
# Filter ADT data to only include hospitalizations in all_ids
adt_final_stitched = adt_stitched[adt_stitched['hospitalization_id'].isin(cohort_df['hospitalization_id'])].copy()
adt_final_stitched = adt_final_stitched.sort_values(by=['encounter_block', 'in_dttm'])
desired_order = ['hospitalization_id', 'encounter_block', 'hospital_id', 'in_dttm', 'out_dttm']
remaining_cols = [col for col in adt_final_stitched.columns if col not in desired_order]
adt_final_stitched = adt_final_stitched[desired_order + remaining_cols]

print("\n=== Validating ICU Administration ===")

adt_final_stitched['is_icu'] = adt_final_stitched['location_category'] == 'icu'

# Check if each hospitalization had at least one ICU stay
hosp_icu_status = adt_final_stitched.groupby('encounter_block')['is_icu'].any()
non_icu_hosps = hosp_icu_status[~hosp_icu_status].index.tolist()
strobe_counts["3_number_hosp_without_ICU_stay"] = len(non_icu_hosps)
print(f"\nNumber of CRRT hospitalizations without any ICU stay: {len(non_icu_hosps)}")
if len(non_icu_hosps) > 0:
    print("WARNING: Found CRRT hospitalizations without ICU stays")
    print("Number of hospitalization IDs without ICU stays:", len(non_icu_hosps), "check crrt_non_icu_df df")
else:
    print("All CRRT hospitalizations had at least one ICU stay")


=== Validating ICU Administration ===

Number of CRRT hospitalizations without any ICU stay: 10
Number of hospitalization IDs without ICU stays: 10 check crrt_non_icu_df df


In [25]:
crrt_non_icu_df = crrt_df[crrt_df['encounter_block'].isin(non_icu_hosps)]
crrt_non_icu_df = crrt_non_icu_df.sort_values(by=['hospitalization_id', 'encounter_block', 'recorded_dttm'])
desired_order = ['hospitalization_id', 'encounter_block', 'recorded_dttm', 'crrt_mode_category']
remaining_cols = [col for col in crrt_non_icu_df.columns if col not in desired_order]
crrt_non_icu_df = crrt_non_icu_df[desired_order + remaining_cols]
crrt_non_icu_df

,hospitalization_id,encounter_block,recorded_dttm,crrt_mode_category,crrt_mode_name,blood_flow_rate,pre_filter_replacement_fluid_rate,post_filter_replacement_fluid_rate,dialysate_flow_rate,ultrafiltration_out
46968,633645555,155665,2021-04-23 16:00:00-05:00,cvvhd,<NA>,250.0,<NA>,<NA>,NaN,NaN
166638,633645555,155665,2021-04-23 17:00:00-05:00,cvvhd,<NA>,250.0,<NA>,<NA>,1280.0,114.0
108606,633645555,155665,2021-04-23 18:00:00-05:00,cvvhd,<NA>,250.0,<NA>,<NA>,1920.0,142.0
128019,633645555,155665,2021-04-23 19:00:00-05:00,cvvhd,<NA>,250.0,<NA>,<NA>,1950.0,145.0
173627,633645555,155665,2021-04-23 20:00:00-05:00,cvvhd,<NA>,250.0,<NA>,<NA>,1910.0,141.0
...,...,...,...,...,...,...,...,...,...,...
130637,645586474,3314,2020-07-09 08:30:00-05:00,cvvhd,<NA>,260.0,<NA>,<NA>,NaN,NaN
119797,645586474,3314,2020-07-09 09:00:00-05:00,cvvhd,<NA>,260.0,<NA>,<NA>,NaN,NaN
75044,645586474,3314,2020-07-09 09:30:00-05:00,cvvhd,<NA>,260.0,<NA>,<NA>,NaN,NaN
9418,645586474,3314,2020-07-09 10:30:00-05:00,cvvhd,<NA>,260.0,<NA>,<NA>,NaN,NaN


In [26]:
adt_subset = adt_df[adt_df['hospitalization_id'].isin(crrt_non_icu_df['hospitalization_id'].unique())]
hosp_subset = hosp_df[hosp_df['hospitalization_id'].isin(crrt_non_icu_df['hospitalization_id'].unique())]

## Labs, Vitals, Meds for the cohort 

In [27]:
cohort_ids_list = list(cohort_df['hospitalization_id'])

In [28]:
# Vitals
vitals_required_columns = [
    'hospitalization_id',
    'recorded_dttm',
    'vital_category',
    'vital_value'
]
vitals_of_interest = ['heart_rate', 'respiratory_rate', 'sbp', 'dbp', 'map', 'spo2', 'weight_kg', 'height_cm']

#Labs
labs_required_columns = [
    'hospitalization_id',
    'lab_result_dttm',
    'lab_category',
    'lab_value',
    'lab_value_numeric'
]
labs_of_interest = ['po2_arterial','pco2_arterial', 'ph_arterial','ph_venous', 'bicarbonate','so2_arterial',
                    'sodium', 'potassium', 'chloride', 'calcium_total', 'magnesium', 'creatinine', 
                    'bun', 'glucose_serum', 'lactate', 'hemoglobin' ]

# Continuous administered meds
meds_required_columns = [
    'hospitalization_id',
    'admin_dttm',
    'med_name',
    'med_category',
    'med_dose',
    'med_dose_unit'
]
meds_of_interest = [
    'norepinephrine', 'epinephrine', 'phenylephrine', 'vasopressin',
    'dopamine', 'angiotensin', 'dobutamine', 'milrinone', 'isoproterenol',
    'propofol', 'midazolam', 'lorazepam', 'dexmedetomidine', 
    'vecuronium', 'rocuronium', 'cisatracurium', 'pancuronium'
]

# ----------------------------------------------------------------------------
# Load Labs
# ----------------------------------------------------------------------------
print(f"\nLoading labs table...")
clif.load_table(
    'labs',
    columns=labs_required_columns,
    filters={
        'hospitalization_id': cohort_ids_list,
        'lab_category': labs_of_interest
    }
)
print(f"   Labs loaded: {len(clif.labs.df):,} rows")
print(f"   Unique lab categories: {clif.labs.df['lab_category'].nunique()}")
print(f"   Unique lab hospitalizations: {clif.labs.df['hospitalization_id'].nunique()}")

# ----------------------------------------------------------------------------
# Load Medication Administration (Continuous)
# ----------------------------------------------------------------------------
print(f"\nLoading medication_admin_continuous table...")
clif.load_table(
    'medication_admin_continuous',
    columns=meds_required_columns,
    filters={
        'hospitalization_id': cohort_ids_list,
        'med_category': meds_of_interest
    }
)
print(f"   Medications loaded: {len(clif.medication_admin_continuous.df):,} rows")
print(f"   Unique medication categories: {clif.medication_admin_continuous.df['med_category'].nunique()}")
print(f"   Unique medication_admin_continuous hospitalizations: {clif.medication_admin_continuous.df['hospitalization_id'].nunique()}")



# ----------------------------------------------------------------------------
# Summary
# ----------------------------------------------------------------------------
print("\n" + "=" * 80)
print("Table Loading Summary")
print("=" * 80)

loaded_tables = clif.get_loaded_tables()
for table_name in loaded_tables:
    table = getattr(clif, table_name)
    print(f"   {table_name}: {len(table.df):,} rows")

print(f"\nCohort size: {len(cohort_hosp_ids):,} hospitalizations")

# Free memory
freed = gc.collect()
print(f"\nFreed {freed} objects from memory")


Loading labs table...
   Labs loaded: 2,035,934 rows
   Unique lab categories: 16
   Unique lab hospitalizations: 2247

Loading medication_admin_continuous table...
   Medications loaded: 972,727 rows
   Unique medication categories: 15
   Unique medication_admin_continuous hospitalizations: 2198

Table Loading Summary
   patient: 97,254 rows
   hospitalization: 166,781 rows
   adt: 425,067 rows
   labs: 2,035,934 rows
   medication_admin_continuous: 972,727 rows
   hospital_diagnosis: 108,880 rows
   crrt_therapy: 403,322 rows

Cohort size: 2,247 hospitalizations

Freed 0 objects from memory


In [29]:
print("Creating wide dataset")
clif.create_wide_dataset(
    hospitalization_ids=list(cohort_df['hospitalization_id']),
    tables_to_load=['vitals', 'labs', 'medication_admin_continuous' ],
    category_filters={
        'vitals': vitals_of_interest,
        'labs': labs_of_interest,
        'medication_admin_continuous': meds_of_interest
    },
    show_progress=True
)

Creating wide dataset
=== WIDE DATASET CREATION STARTED ===

Phase 1: Initialization
  1.1: Validating parameters
  1.2: Configuring encounter stitching (enabled)

Phase 2: Encounter Processing
  2.1: === SPECIAL: ENCOUNTER STITCHING ===

Phase 3: Table Loading
  3.1: Auto-loading base tables
  3.2: Loading optional tables: ['vitals', 'labs', 'medication_admin_continuous']
       - Loading vitals table...
       - labs table already loaded
       - medication_admin_continuous table already loaded

Phase 4: Calling Wide Dataset Utility
  4.1: Passing to wide_dataset.create_wide_dataset()
       - Tables: ['vitals', 'labs', 'medication_admin_continuous']
       - Category filters: ['vitals', 'labs', 'medication_admin_continuous']
       - Batch size: 1000
       - Memory limit: None
       - Show progress: True

Phase 4: Wide Dataset Processing (utility function)
  4.1: Starting wide dataset creation...
Filtering to specific hospitalization IDs: 2247 encounters

Loading and filtering bas

Processing batches:   0%|          | 0/3 [00:00<?, ?it/s]

    4.B.1: Processing batch 1/3
             - 1000 hospitalizations in batch

    4.S.1: Loading and filtering base tables
           - Base cohort created with 1000 records
    4.S.3: Processing tables
           - Processing vitals...
Loaded 3476830 records from vitals
           === PIVOTING VITALS ===
           - Categories to pivot: ['heart_rate', 'respiratory_rate', 'sbp', 'dbp', 'map', 'spo2', 'weight_kg', 'height_cm']
Filtering vitals categories to: ['heart_rate', 'respiratory_rate', 'sbp', 'dbp', 'map', 'spo2', 'weight_kg', 'height_cm']
Pivoted vitals: 609233 combo_ids with 8 category columns
           - Processing labs...
Loaded 901976 records from labs
           === PIVOTING LABS ===
           - Categories to pivot: ['po2_arterial', 'pco2_arterial', 'ph_arterial', 'ph_venous', 'bicarbonate', 'so2_arterial', 'sodium', 'potassium', 'chloride', 'calcium_total', 'magnesium', 'creatinine', 'bun', 'glucose_serum', 'lactate', 'hemoglobin']
Filtering labs categories to: ['po2_a

Processing batches:  33%|███▎      | 1/3 [00:07<00:14,  7.08s/it]

    === SPECIAL: MISSING COLUMNS ===
           - Added missing column: vecuronium
           - Added missing column: pancuronium

    4.S.6: Final cleanup
           - Removing duplicate columns
           - Dropping temporary columns (combo_id, date)
           - Wide dataset created: 878397 records with 54 columns
             - Batch 1 completed: 878397 records
    4.B.2: Processing batch 2/3
             - 1000 hospitalizations in batch

    4.S.1: Loading and filtering base tables
           - Base cohort created with 1000 records
    4.S.3: Processing tables
           - Processing vitals...
Loaded 3738375 records from vitals
           === PIVOTING VITALS ===
           - Categories to pivot: ['heart_rate', 'respiratory_rate', 'sbp', 'dbp', 'map', 'spo2', 'weight_kg', 'height_cm']
Filtering vitals categories to: ['heart_rate', 'respiratory_rate', 'sbp', 'dbp', 'map', 'spo2', 'weight_kg', 'height_cm']
Pivoted vitals: 670547 combo_ids with 8 category columns
           - Processi

Processing batches:  67%|██████▋   | 2/3 [00:12<00:06,  6.03s/it]

    === SPECIAL: MISSING COLUMNS ===
           - Added missing column: vecuronium
           - Added missing column: pancuronium

    4.S.6: Final cleanup
           - Removing duplicate columns
           - Dropping temporary columns (combo_id, date)
           - Wide dataset created: 944391 records with 54 columns
             - Batch 2 completed: 944391 records
    4.B.3: Processing batch 3/3
             - 247 hospitalizations in batch

    4.S.1: Loading and filtering base tables
           - Base cohort created with 247 records
    4.S.3: Processing tables
           - Processing vitals...
Loaded 884569 records from vitals
           === PIVOTING VITALS ===
           - Categories to pivot: ['heart_rate', 'respiratory_rate', 'sbp', 'dbp', 'map', 'spo2', 'weight_kg', 'height_cm']
Filtering vitals categories to: ['heart_rate', 'respiratory_rate', 'sbp', 'dbp', 'map', 'spo2', 'weight_kg', 'height_cm']
Pivoted vitals: 157157 combo_ids with 8 category columns
           - Processing 

Processing batches: 100%|██████████| 3/3 [00:14<00:00,  4.97s/it]

    === SPECIAL: MISSING COLUMNS ===
           - Added missing column: vecuronium
           - Added missing column: pancuronium

    4.S.6: Final cleanup
           - Removing duplicate columns
           - Dropping temporary columns (combo_id, date)
           - Wide dataset created: 221933 records with 54 columns
             - Batch 3 completed: 221933 records
             - Combining 3 batch results...
             - Final dataset: 2044721 records with 54 columns



Phase 5: Post-Processing
  5.1: === SPECIAL: ADDING ENCOUNTER BLOCKS ===
       - Encounter_block column already present - 2247 unique encounter blocks
  5.2: No assessment type optimization needed

Phase 6: Completion
  6.1: Wide dataset stored in self.wide_df
  6.2: Dataset shape: 2044721 rows x 54 columns

=== WIDE DATASET CREATION COMPLETED ===



In [30]:
vars_to_keep = [
       'patient_id','hospitalization_id', 'event_time',
       'angiotensin', 'cisatracurium', 'dexmedetomidine', 'dobutamine',
       'dopamine', 'epinephrine', 'isoproterenol', 'lorazepam', 'midazolam',
       'milrinone', 'norepinephrine', 'phenylephrine', 'propofol',
       'rocuronium', 'vasopressin', 'bicarbonate', 'bun', 'calcium_total',
       'chloride', 'creatinine', 'glucose_serum', 'hemoglobin', 'lactate',
       'magnesium', 'pco2_arterial', 'ph_arterial', 'ph_venous',
       'po2_arterial', 'potassium', 'so2_arterial', 'sodium', 'dbp',
       'heart_rate', 'height_cm', 'map', 'respiratory_rate', 'sbp', 'spo2',
       'weight_kg','vecuronium', 'pancuronium'
]
clif.wide_df = clif.wide_df.loc[:, vars_to_keep]
clif.wide_df.rename(columns={'event_time': 'recorded_dttm'}, inplace=True)
wide_df = clif.wide_df
wide_df = wide_df.merge(
    encounter_mapping,
    on = 'hospitalization_id',
     how='left'
)

In [31]:
wide_df.dtypes

patient_id                                object
hospitalization_id                        object
recorded_dttm         datetime64[us, US/Central]
angiotensin                              float64
cisatracurium                            float64
dexmedetomidine                          float64
dobutamine                               float64
dopamine                                 float64
epinephrine                              float64
isoproterenol                            float64
lorazepam                                float64
midazolam                                float64
milrinone                                float64
norepinephrine                           float64
phenylephrine                            float64
propofol                                 float64
rocuronium                               float64
vasopressin                              float64
bicarbonate                              float64
bun                                      float64
calcium_total       

## Check PF ratio calc 

In [32]:
clif.load_table('respiratory_support',
                        columns=rst_required_columns,
                        filters={'hospitalization_id': cohort_ids_list})

In [33]:
resp_support = clif.respiratory_support.df

In [34]:
merged_df = wide_df.merge(
    resp_support,
    on=['hospitalization_id', 'recorded_dttm'],
    how='outer'
)
merged_df.dtypes

patient_id                                           object
hospitalization_id                                   object
recorded_dttm                    datetime64[us, US/Central]
angiotensin                                         float64
cisatracurium                                       float64
                                            ...            
pressure_support_set                                float64
peak_inspiratory_pressure_set                       float64
peak_inspiratory_pressure_obs                       float64
plateau_pressure_obs                                float64
minute_vent_obs                                     float64
Length: 62, dtype: object

In [35]:
import duckdb
q = f"""
        FROM merged_df
        SELECT *
        WHERE fio2_set IS NOT NULL OR po2_arterial IS NOT NULL OR device_category IS NOT NULL
        """
df = duckdb.sql(q).df()

In [36]:
df_subset = df[['hospitalization_id', 
                'recorded_dttm','po2_arterial','fio2_set', 'device_category',
                'mode_category'
]].sort_values(['hospitalization_id', 'recorded_dttm'])

## Outcomes

In [37]:
# ICU SEGMENTS - equivalent to icu_segs in R
print("\n1. Processing ICU segments...")
icu_segs = adt_final_stitched.copy()
icu_segs['loccat'] = icu_segs['location_category'].str.lower()
icu_segs = icu_segs[
    (icu_segs['loccat'] == 'icu') &
    (icu_segs['in_dttm'].notna()) &
    (icu_segs['out_dttm'].notna()) &
    (icu_segs['out_dttm'] > icu_segs['in_dttm'])
]

print(f"   ICU segments identified: {len(icu_segs):,}")

# 2. ICU LENGTH OF STAY 
print("\n2. Calculating ICU Length of Stay...")
icu_los = icu_segs[icu_segs['hospitalization_id'].isin(cohort_df['hospitalization_id'])].copy()
icu_los['seg_days'] = (icu_los['out_dttm'] - icu_los['in_dttm']).dt.total_seconds() / (24 * 3600)

icu_los_summary = icu_los.groupby('hospitalization_id').agg({
    'seg_days': 'sum'
}).reset_index()
icu_los_summary.rename(columns={'seg_days': 'icu_los_days'}, inplace=True)

print(f"   Hospitalizations with ICU LOS calculated: {len(icu_los_summary):,}")
print(f"   Median ICU LOS: {icu_los_summary['icu_los_days'].median():.2f} days")

# VITALS DATETIME RANGE -
print("\n3. Calculating vitals datetime range...")
# Use the wide_df which contains vitals data
vitals_dttm = wide_df[wide_df['hospitalization_id'].isin(cohort_df['hospitalization_id'])].copy()
vitals_dttm = vitals_dttm[vitals_dttm['recorded_dttm'].notna()]

vitals_range = vitals_dttm.groupby('hospitalization_id').agg({
    'recorded_dttm': ['min', 'max']
}).reset_index()
vitals_range.columns = ['hospitalization_id', 'first_vital_dttm', 'last_vital_dttm']

print(f"   Hospitalizations with vitals range: {len(vitals_range):,}")

# HOSPITAL LENGTH OF STAY - equivalent to hosp_los in R
print("\n4. Calculating Hospital Length of Stay...")
hosp_los = cohort_df[['hospitalization_id']].merge(
    vitals_range, 
    on='hospitalization_id', 
    how='left'
)

# Calculate hospital LOS in days
hosp_los['hosp_los_days'] = (
    hosp_los['last_vital_dttm'] - hosp_los['first_vital_dttm']
).dt.total_seconds() / (24 * 3600)

# Ensure non-negative values and handle infinities
hosp_los['hosp_los_days'] = hosp_los['hosp_los_days'].apply(
    lambda x: max(x, 0) if pd.notna(x) and np.isfinite(x) else np.nan
)

hospitalization_df = clif.hospitalization.df
# Merge with hospitalization data
hosp_los = hosp_los.merge(
    hospitalization_df[['hospitalization_id', 'discharge_category']],
    on='hospitalization_id',
    how='left'
)

print(f"   Hospital LOS calculated for: {hosp_los['hosp_los_days'].notna().sum():,} hospitalizations")
print(f"   Median Hospital LOS: {hosp_los['hosp_los_days'].median():.2f} days")

# 5. FINAL OUTCOME TIMES - equivalent to final_outcome_times in R
print("\n5. Processing final outcome times...")
final_outcome_times = hospitalization_df[
    ['patient_id', 'hospitalization_id', 'discharge_category', 'discharge_dttm']
].copy()

final_outcome_times = final_outcome_times[
    final_outcome_times['hospitalization_id'].isin(cohort_df['hospitalization_id'])
]

final_outcome_times['discharge_cat_low'] = final_outcome_times['discharge_category'].str.lower()

# Merge with patient death data
patient_df = clif.patient.df[['patient_id', 'death_dttm']]
final_outcome_times = final_outcome_times.merge(
    patient_df, 
    on='patient_id', 
    how='left'
)

# Merge with vitals range
final_outcome_times = final_outcome_times.merge(
    vitals_range, 
    on='hospitalization_id', 
    how='left'
)

# Create final death datetime - use last vital time if discharged as expired/hospice but no death_dttm
final_outcome_times['death_dttm_final'] = final_outcome_times.apply(
    lambda row: row['last_vital_dttm'] 
    if (row['discharge_cat_low'] in ['expired', 'hospice'] and pd.isna(row['death_dttm']))
    else row['death_dttm'],
    axis=1
)

print(f"   Final outcome times processed for: {len(final_outcome_times):,} hospitalizations")

# 6. MORTALITY CALCULATIONS - equivalent to mortality_instay in R
print("\n6. Calculating mortality outcomes...")

# Need to get admission and discharge times - merge with hospitalization data
mortality_base = cohort_df.merge(
    hospitalization_df[['hospitalization_id', 'admission_dttm', 'discharge_dttm']],
    on='hospitalization_id',
    how='left'
)

mortality_instay = mortality_base.merge(
    final_outcome_times[['hospitalization_id', 'death_dttm_final']],
    on='hospitalization_id',
    how='left'
)

# Calculate mortality flags
mortality_instay['death_ts'] = mortality_instay['death_dttm_final']

# In-hospital death: death occurred between admission and discharge
mortality_instay['in_hosp_death'] = (
    (mortality_instay['death_ts'].notna()) &
    (mortality_instay['death_ts'] >= mortality_instay['admission_dttm']) &
    (mortality_instay['death_ts'] <= mortality_instay['discharge_dttm'])
).astype(int)

# 30-day mortality: death within 30 days of admission
mortality_instay['death_30d'] = (
    (mortality_instay['death_ts'].notna()) &
    (mortality_instay['death_ts'] <= (mortality_instay['admission_dttm'] + pd.Timedelta(days=30)))
).astype(int)

# Final mortality dataset
mortality_final = mortality_instay[['hospitalization_id', 'in_hosp_death', 'death_30d']].copy()

print(f"   Mortality calculated for: {len(mortality_final):,} hospitalizations")
print(f"   In-hospital deaths: {mortality_final['in_hosp_death'].sum():,} ({mortality_final['in_hosp_death'].mean()*100:.1f}%)")
print(f"   30-day deaths: {mortality_final['death_30d'].sum():,} ({mortality_final['death_30d'].mean()*100:.1f}%)")

# 7. COMBINE ALL OUTCOMES
print("\n7. Combining all outcomes...")
outcomes_df = cohort_df[['hospitalization_id', 'encounter_block']].merge(
    icu_los_summary, on='hospitalization_id', how='left'
).merge(
    hosp_los[['hospitalization_id', 'hosp_los_days']], on='hospitalization_id', how='left'
).merge(
    mortality_final, on='hospitalization_id', how='left'
)

print(f"\nFinal outcomes dataset:")
print(f"   Total records: {len(outcomes_df):,}")
print(f"   Records with ICU LOS: {outcomes_df['icu_los_days'].notna().sum():,}")
print(f"   Records with Hospital LOS: {outcomes_df['hosp_los_days'].notna().sum():,}")
print(f"   In-hospital mortality rate: {outcomes_df['in_hosp_death'].mean()*100:.1f}%")
print(f"   30-day mortality rate: {outcomes_df['death_30d'].mean()*100:.1f}%")


1. Processing ICU segments...
   ICU segments identified: 4,696

2. Calculating ICU Length of Stay...
   Hospitalizations with ICU LOS calculated: 2,237
   Median ICU LOS: 7.28 days

3. Calculating vitals datetime range...
   Hospitalizations with vitals range: 2,247

4. Calculating Hospital Length of Stay...
   Hospital LOS calculated for: 2,247 hospitalizations
   Median Hospital LOS: 14.60 days

5. Processing final outcome times...
   Final outcome times processed for: 2,247 hospitalizations

6. Calculating mortality outcomes...
   Mortality calculated for: 2,247 hospitalizations
   In-hospital deaths: 1,296 (57.7%)
   30-day deaths: 1,338 (59.5%)

7. Combining all outcomes...

Final outcomes dataset:
   Total records: 2,247
   Records with ICU LOS: 2,237
   Records with Hospital LOS: 2,247
   In-hospital mortality rate: 57.7%
   30-day mortality rate: 59.5%


In [38]:
outcomes_df.dtypes

hospitalization_id     object
encounter_block         int32
icu_los_days          float64
hosp_los_days         float64
in_hosp_death           int64
death_30d               int64
dtype: object

## Process CRRT

In [39]:
# Filter crrt_df to only include hospitalization_ids present in the cohort
crrt_cohort = crrt_df[crrt_df['hospitalization_id'].isin(cohort_df['hospitalization_id'])].copy()
# del crrt_df

#assume on_crrt for all rows in the crrt table 
crrt_cohort.loc[:, "on_crrt"] = 1

# Forward fill blood_flow_rate within each encounter_block
crrt_cohort = crrt_cohort.sort_values(['hospitalization_id', 'encounter_block', 'recorded_dttm'])
crrt_cohort['blood_flow_rate'] = crrt_cohort.groupby('encounter_block')['blood_flow_rate'].ffill()

In [40]:
crrt_cohort.dtypes

hospitalization_id                                string[python]
recorded_dttm                         datetime64[us, US/Central]
crrt_mode_name                                             Int32
crrt_mode_category                                        object
blood_flow_rate                                          float64
pre_filter_replacement_fluid_rate                          Int32
post_filter_replacement_fluid_rate                         Int32
dialysate_flow_rate                                      float64
ultrafiltration_out                                      float64
encounter_block                                            int32
on_crrt                                                    int64
dtype: object

In [41]:
print("\n" + "=" * 80)
print("Processing CRRT Data for ATS 2026 Study")
print("=" * 80)

# Step 3: Calculate Index Time (First CRRT per encounter_block)
print("\n3. Calculating Index Time...")
index_times = crrt_cohort.groupby('encounter_block').agg({
    'recorded_dttm': 'min'
}).reset_index()
index_times.rename(columns={'recorded_dttm': 'index_time'}, inplace=True)

print(f"   Index times calculated for: {len(index_times):,} encounter_blocks")

# Step 4: Get Weight Data - SIMPLE APPROACH
print("\n4. Finding closest weights to index time...")

# Extract weight data
weight_data = wide_df[['encounter_block', 'recorded_dttm', 'weight_kg']].copy()
weight_data = weight_data[weight_data['weight_kg'].notna()]

print(f"   Weight records available: {len(weight_data):,}")

# Simple merge and filter approach
combined = index_times.merge(weight_data, on='encounter_block', how='inner')

# Keep only weights recorded before or at index time
combined = combined[combined['recorded_dttm'] <= combined['index_time']]

# For each encounter_block, get the weight closest to (but not after) index_time
closest_weights = (combined
                  .sort_values(['encounter_block', 'recorded_dttm'])
                  .groupby('encounter_block')
                  .last()
                  .reset_index())

# Keep only the columns we need
closest_weights = closest_weights[['encounter_block', 'weight_kg']]

print(f"   Weights found for: {len(closest_weights):,} encounter_blocks")

# Step 5: Get CRRT parameters at index time
print("\n5. Getting CRRT parameters at index time...")

# Merge CRRT data with index times
crrt_with_index = crrt_cohort.merge(
    index_times, 
    on='encounter_block',
    how='inner'
)


# Filter to records at exactly the index time
crrt_at_index = crrt_with_index[crrt_with_index['recorded_dttm'] == crrt_with_index['index_time']].copy()

print(f"   CRRT records at index time: {len(crrt_at_index):,}")

# Step 6: Combine CRRT data with weights
print("\n6. Combining results...")

index_crrt_df = crrt_at_index.merge(
    closest_weights, 
    on='encounter_block', 
    how='left'
)

print(f"   Final dataset: {len(index_crrt_df):,} records")
print(f"   Records with weights: {index_crrt_df['weight_kg'].notna().sum():,}")
print(f"   Records with CRRT mode: {index_crrt_df['crrt_mode_category'].notna().sum():,}")

# Verify what we have
print("\n7. Data validation...")
print(f"   Unique encounter blocks: {index_crrt_df['encounter_block'].nunique():,}")
print(f"   Date range: {index_crrt_df['index_time'].min()} to {index_crrt_df['index_time'].max()}")


Processing CRRT Data for ATS 2026 Study

3. Calculating Index Time...
   Index times calculated for: 2,247 encounter_blocks

4. Finding closest weights to index time...
   Weight records available: 45,006
   Weights found for: 2,190 encounter_blocks

5. Getting CRRT parameters at index time...
   CRRT records at index time: 2,247

6. Combining results...
   Final dataset: 2,247 records
   Records with weights: 2,190
   Records with CRRT mode: 2,247

7. Data validation...
   Unique encounter blocks: 2,247
   Date range: 2018-01-06 15:15:00-06:00 to 2024-12-31 01:00:00-06:00


In [42]:
index_crrt_df.dtypes

hospitalization_id                                string[python]
recorded_dttm                         datetime64[us, US/Central]
crrt_mode_name                                             Int32
crrt_mode_category                                        object
blood_flow_rate                                          float64
pre_filter_replacement_fluid_rate                          Int32
post_filter_replacement_fluid_rate                         Int32
dialysate_flow_rate                                      float64
ultrafiltration_out                                      float64
encounter_block                                            int32
on_crrt                                                    int64
index_time                            datetime64[us, US/Central]
weight_kg                                                float64
dtype: object

In [43]:
# ============================================================================
# STEP 7: Calculate CRRT Dose
# ============================================================================
print("\n" + "=" * 80)
print("Step 7: Calculating CRRT Dose at Index Time")
print("=" * 80)

# Fill NaN values with 0 for flow rate calculations
flow_cols = ['dialysate_flow_rate', 'pre_filter_replacement_fluid_rate', 'post_filter_replacement_fluid_rate']
index_crrt_df[flow_cols] = index_crrt_df[flow_cols].fillna(0)

# Standardize mode category to lowercase
index_crrt_df['crrt_mode_category'] = index_crrt_df['crrt_mode_category'].str.lower()

print("\n   Mode distribution:")
print(index_crrt_df['crrt_mode_category'].value_counts())

# Calculate total flow based on mode (vectorized approach)
print("\n   Calculating mode-specific flow rates...")

conditions = [
    index_crrt_df['crrt_mode_category'] == 'cvvhd',
    index_crrt_df['crrt_mode_category'] == 'cvvh', 
    index_crrt_df['crrt_mode_category'] == 'cvvhdf'
]

choices = [
    # CVVHD: Dialysate flow rate alone
    index_crrt_df['dialysate_flow_rate'],
    # CVVH: Replacement fluid rate (pre + post)
    index_crrt_df['pre_filter_replacement_fluid_rate'] + index_crrt_df['post_filter_replacement_fluid_rate'],
    # CVVHDF: All flows combined
    index_crrt_df['dialysate_flow_rate'] + index_crrt_df['pre_filter_replacement_fluid_rate'] + index_crrt_df['post_filter_replacement_fluid_rate']
]

# Apply vectorized conditions
index_crrt_df['total_flow_rate'] = np.select(conditions, choices, default=np.nan)

print(f"   Total flow rates calculated: {index_crrt_df['total_flow_rate'].notna().sum():,}")

# Calculate dose: total_flow_rate / weight_kg (vectorized)
print("\n   Calculating CRRT dose (mL/kg/hr)...")

index_crrt_df['crrt_dose_ml_kg_hr'] = np.where(
    (index_crrt_df['weight_kg'] > 0) & (index_crrt_df['total_flow_rate'] > 0),
    index_crrt_df['total_flow_rate'] / index_crrt_df['weight_kg'],
    np.nan
)

# ============================================================================
# STEP 8: Summary Statistics
# ============================================================================
print("\n" + "=" * 80)
print("Step 8: CRRT Dose Summary Statistics")
print("=" * 80)

dose_available = index_crrt_df['crrt_dose_ml_kg_hr'].notna()
print(f"\n   CRRT doses calculated: {dose_available.sum():,}/{len(index_crrt_df):,} ({dose_available.mean()*100:.1f}%)")

if dose_available.sum() > 0:
    dose_stats = index_crrt_df['crrt_dose_ml_kg_hr'].describe()
    print(f"\n   Dose statistics (mL/kg/hr):")
    print(f"     Mean ± SD: {dose_stats['mean']:.1f} ± {dose_stats['std']:.1f}")
    print(f"     Median [IQR]: {dose_stats['50%']:.1f} [{dose_stats['25%']:.1f}-{dose_stats['75%']:.1f}]")
    print(f"     Range: {dose_stats['min']:.1f} - {dose_stats['max']:.1f}")
    
    # Clinical targets analysis
    print(f"\n   Clinical Target Analysis:")
    target_range = ((index_crrt_df['crrt_dose_ml_kg_hr'] >= 20) & 
                   (index_crrt_df['crrt_dose_ml_kg_hr'] <= 25)).sum()
    below_target = (index_crrt_df['crrt_dose_ml_kg_hr'] < 20).sum()
    above_target = (index_crrt_df['crrt_dose_ml_kg_hr'] > 25).sum()
    
    print(f"     Below target (<20 mL/kg/hr): {below_target:,} ({below_target/dose_available.sum()*100:.1f}%)")
    print(f"     Within target (20-25 mL/kg/hr): {target_range:,} ({target_range/dose_available.sum()*100:.1f}%)")
    print(f"     Above target (>25 mL/kg/hr): {above_target:,} ({above_target/dose_available.sum()*100:.1f}%)")
    
    # Edge case detection
    print(f"\n   Edge Case Detection:")
    extreme_low = (index_crrt_df['crrt_dose_ml_kg_hr'] < 10).sum()
    extreme_high = (index_crrt_df['crrt_dose_ml_kg_hr'] > 50).sum()
    
    if extreme_low > 0:
        print(f"     ⚠️  Extreme low doses (<10 mL/kg/hr): {extreme_low}")
    if extreme_high > 0:
        print(f"     ⚠️  Extreme high doses (>50 mL/kg/hr): {extreme_high}")

# Mode breakdown with dose statistics
print(f"\n   Dose by CRRT Mode:")
for mode in index_crrt_df['crrt_mode_category'].unique():
    if pd.notna(mode):
        mode_data = index_crrt_df[index_crrt_df['crrt_mode_category'] == mode]
        mode_doses = mode_data['crrt_dose_ml_kg_hr'].dropna()
        
        if len(mode_doses) > 0:
            print(f"     {mode.upper()}:")
            print(f"       Count: {len(mode_data):,} ({len(mode_data)/len(index_crrt_df)*100:.1f}%)")
            print(f"       Doses available: {len(mode_doses):,}")
            print(f"       Mean dose: {mode_doses.mean():.1f} mL/kg/hr")
            print(f"       Median dose: {mode_doses.median():.1f} mL/kg/hr")

# ============================================================================
# STEP 9: Create Final Analysis Dataset
# ============================================================================
print("\n" + "=" * 80)
print("Step 9: Creating Final Analysis Dataset")
print("=" * 80)

# Select key columns for analysis
analysis_columns = [
    'encounter_block', 
    'hospitalization_id',
    'index_time',
    'crrt_mode_category',
    'weight_kg',
    'total_flow_rate',
    'crrt_dose_ml_kg_hr',
    'dialysate_flow_rate',
    'pre_filter_replacement_fluid_rate',
    'post_filter_replacement_fluid_rate',
    'blood_flow_rate',
    'ultrafiltration_out'
]

# Create final analysis dataset
crrt_analysis_df = index_crrt_df[analysis_columns].copy()

print(f"\n   Final analysis dataset created:")
print(f"     Total records: {len(crrt_analysis_df):,}")
print(f"     Unique encounter blocks: {crrt_analysis_df['encounter_block'].nunique():,}")
print(f"     Records with valid dose: {crrt_analysis_df['crrt_dose_ml_kg_hr'].notna().sum():,}")
print(f"     Records with weight: {crrt_analysis_df['weight_kg'].notna().sum():,}")

# Quality checks
print(f"\n   Quality Checks:")
print(f"     Missing weights: {crrt_analysis_df['weight_kg'].isna().sum():,}")
print(f"     Missing doses: {crrt_analysis_df['crrt_dose_ml_kg_hr'].isna().sum():,}")
print(f"     Missing mode: {crrt_analysis_df['crrt_mode_category'].isna().sum():,}")

print("\n✅ CRRT dose calculation completed successfully!")
print(f"   Dataset 'crrt_analysis_df' ready for propensity score analysis")


Step 7: Calculating CRRT Dose at Index Time

   Mode distribution:
crrt_mode_category
cvvhd    2207
scuf       40
Name: count, dtype: int64

   Calculating mode-specific flow rates...
   Total flow rates calculated: 2,207

   Calculating CRRT dose (mL/kg/hr)...

Step 8: CRRT Dose Summary Statistics

   CRRT doses calculated: 776/2,247 (34.5%)

   Dose statistics (mL/kg/hr):
     Mean ± SD: 25.2 ± 53.1
     Median [IQR]: 19.1 [9.0-30.3]
     Range: 0.1 - 1012.6

   Clinical Target Analysis:
     Below target (<20 mL/kg/hr): 409 (52.7%)
     Within target (20-25 mL/kg/hr): 76 (9.8%)
     Above target (>25 mL/kg/hr): 291 (37.5%)

   Edge Case Detection:
     ⚠️  Extreme low doses (<10 mL/kg/hr): 221
     ⚠️  Extreme high doses (>50 mL/kg/hr): 57

   Dose by CRRT Mode:
     CVVHD:
       Count: 2,207 (98.2%)
       Doses available: 776
       Mean dose: 25.2 mL/kg/hr
       Median dose: 19.1 mL/kg/hr

Step 9: Creating Final Analysis Dataset

   Final analysis dataset created:
     Total r

In [44]:
# ============================================================================
# STEP 10: Time-Weighted Average Dose Analysis
# ============================================================================
print("\n" + "=" * 80)
print("Step 10: Time-Weighted Average CRRT Dose Analysis")
print("=" * 80)

# Get all CRRT records with index time and encounter blocks
crrt_with_index = crrt_cohort.merge(
    index_times,
    on='encounter_block',
    how='inner'
)

# Calculate time from index
crrt_with_index['hours_from_index'] = (
    crrt_with_index['recorded_dttm'] - crrt_with_index['index_time']
).dt.total_seconds() / 3600

# Filter for records within time windows
crrt_6h = crrt_with_index[
    (crrt_with_index['hours_from_index'] >= 0) & 
    (crrt_with_index['hours_from_index'] <= 6)
].copy()

crrt_24h = crrt_with_index[
    (crrt_with_index['hours_from_index'] >= 0) & 
    (crrt_with_index['hours_from_index'] <= 24)
].copy()

print(f"\n   CRRT records by time window:")
print(f"     At initiation (t=0): {len(crrt_at_index):,} records")
print(f"     Within 6 hours: {len(crrt_6h):,} records")
print(f"     Within 24 hours: {len(crrt_24h):,} records")

# ============================================================================
# Calculate instantaneous doses for each time window
# ============================================================================

def calculate_crrt_dose(df, closest_weights):
    """Calculate CRRT dose for a dataframe of CRRT records"""
    # Merge with weights
    df_with_weight = df.merge(
        closest_weights[['encounter_block', 'weight_kg']],
        on='encounter_block',
        how='left'
    )
    
    # Fill NaN flow rates with 0
    flow_cols = ['dialysate_flow_rate', 'pre_filter_replacement_fluid_rate', 
                 'post_filter_replacement_fluid_rate']
    df_with_weight[flow_cols] = df_with_weight[flow_cols].fillna(0)
    
    # Standardize mode
    df_with_weight['crrt_mode_category'] = df_with_weight['crrt_mode_category'].str.lower()
    
    # Calculate total flow based on mode
    conditions = [
        df_with_weight['crrt_mode_category'] == 'cvvhd',
        df_with_weight['crrt_mode_category'] == 'cvvh',
        df_with_weight['crrt_mode_category'] == 'cvvhdf'
    ]
    
    choices = [
        df_with_weight['dialysate_flow_rate'],
        df_with_weight['pre_filter_replacement_fluid_rate'] + df_with_weight['post_filter_replacement_fluid_rate'],
        df_with_weight['dialysate_flow_rate'] + df_with_weight['pre_filter_replacement_fluid_rate'] + df_with_weight['post_filter_replacement_fluid_rate']
    ]
    
    df_with_weight['total_flow_rate'] = np.select(conditions, choices, default=np.nan)
    
    # Calculate dose
    df_with_weight['crrt_dose_ml_kg_hr'] = np.where(
        (df_with_weight['weight_kg'] > 0) & (df_with_weight['total_flow_rate'] > 0),
        df_with_weight['total_flow_rate'] / df_with_weight['weight_kg'],
        np.nan
    )
    
    return df_with_weight

# Calculate doses for each window
print("\n   Calculating doses for time windows...")
crrt_6h_doses = calculate_crrt_dose(crrt_6h, closest_weights)
crrt_24h_doses = calculate_crrt_dose(crrt_24h, closest_weights)

# ============================================================================
# Calculate time-weighted averages
# ============================================================================

def calculate_time_weighted_dose(df):
    """
    Calculate time-weighted average dose per encounter block.
    Uses trapezoidal rule for integration.
    """
    results = []
    
    for block, group in df.groupby('encounter_block'):
        # Sort by time
        group = group.sort_values('recorded_dttm')
        
        # Filter out records without valid doses
        valid_doses = group[group['crrt_dose_ml_kg_hr'].notna()].copy()
        
        if len(valid_doses) == 0:
            results.append({
                'encounter_block': block,
                'time_weighted_dose': np.nan,
                'n_measurements': 0,
                'duration_hours': 0
            })
            continue
        
        if len(valid_doses) == 1:
            # Only one measurement - use it as the average
            results.append({
                'encounter_block': block,
                'time_weighted_dose': valid_doses['crrt_dose_ml_kg_hr'].iloc[0],
                'n_measurements': 1,
                'duration_hours': 0
            })
            continue
        
        # Calculate time-weighted average using trapezoidal rule
        times = valid_doses['hours_from_index'].values
        doses = valid_doses['crrt_dose_ml_kg_hr'].values
        
        # Calculate weighted sum
        weighted_sum = 0
        for i in range(len(times) - 1):
            dt = times[i+1] - times[i]
            avg_dose = (doses[i] + doses[i+1]) / 2
            weighted_sum += avg_dose * dt
        
        total_duration = times[-1] - times[0]
        time_weighted_dose = weighted_sum / total_duration if total_duration > 0 else doses.mean()
        
        results.append({
            'encounter_block': block,
            'time_weighted_dose': time_weighted_dose,
            'n_measurements': len(valid_doses),
            'duration_hours': total_duration
        })
    
    return pd.DataFrame(results)

print("\n   Calculating time-weighted averages...")
dose_6h_summary = calculate_time_weighted_dose(crrt_6h_doses)
dose_24h_summary = calculate_time_weighted_dose(crrt_24h_doses)

# ============================================================================
# Summary Statistics
# ============================================================================
print("\n" + "=" * 80)
print("CRRT Dose Comparison: Initiation vs 6h vs 24h")
print("=" * 80)

# At initiation
init_available = crrt_analysis_df['crrt_dose_ml_kg_hr'].notna().sum()
print(f"\n📍 At Initiation (t=0):")
print(f"   Doses available: {init_available:,}/{len(crrt_analysis_df):,} ({init_available/len(crrt_analysis_df)*100:.1f}%)")
if init_available > 0:
    init_stats = crrt_analysis_df['crrt_dose_ml_kg_hr'].describe()
    print(f"   Mean ± SD: {init_stats['mean']:.1f} ± {init_stats['std']:.1f} mL/kg/hr")
    print(f"   Median [IQR]: {init_stats['50%']:.1f} [{init_stats['25%']:.1f}-{init_stats['75%']:.1f}]")

# 6 hours
dose_6h_available = dose_6h_summary['time_weighted_dose'].notna().sum()
print(f"\n⏰ First 6 Hours:")
print(f"   Doses available: {dose_6h_available:,}/{len(dose_6h_summary):,} ({dose_6h_available/len(dose_6h_summary)*100:.1f}%)")
if dose_6h_available > 0:
    stats_6h = dose_6h_summary['time_weighted_dose'].describe()
    print(f"   Mean ± SD: {stats_6h['mean']:.1f} ± {stats_6h['std']:.1f} mL/kg/hr")
    print(f"   Median [IQR]: {stats_6h['50%']:.1f} [{stats_6h['25%']:.1f}-{stats_6h['75%']:.1f}]")
    print(f"   Avg measurements per encounter: {dose_6h_summary['n_measurements'].mean():.1f}")

# 24 hours
dose_24h_available = dose_24h_summary['time_weighted_dose'].notna().sum()
print(f"\n📅 First 24 Hours:")
print(f"   Doses available: {dose_24h_available:,}/{len(dose_24h_summary):,} ({dose_24h_available/len(dose_24h_summary)*100:.1f}%)")
if dose_24h_available > 0:
    stats_24h = dose_24h_summary['time_weighted_dose'].describe()
    print(f"   Mean ± SD: {stats_24h['mean']:.1f} ± {stats_24h['std']:.1f} mL/kg/hr")
    print(f"   Median [IQR]: {stats_24h['50%']:.1f} [{stats_24h['25%']:.1f}-{stats_24h['75%']:.1f}]")
    print(f"   Avg measurements per encounter: {dose_24h_summary['n_measurements'].mean():.1f}")

# ============================================================================
# Clinical Target Analysis by Time Window
# ============================================================================
print("\n" + "=" * 80)
print("Clinical Target Analysis (20-25 mL/kg/hr)")
print("=" * 80)

def analyze_targets(doses, label):
    """Analyze how many doses meet clinical targets"""
    valid_doses = doses[doses.notna()]
    if len(valid_doses) == 0:
        return
    
    below = (valid_doses < 20).sum()
    within = ((valid_doses >= 20) & (valid_doses <= 25)).sum()
    above = (valid_doses > 25).sum()
    
    print(f"\n{label}:")
    print(f"   Below target (<20): {below:,} ({below/len(valid_doses)*100:.1f}%)")
    print(f"   Within target (20-25): {within:,} ({within/len(valid_doses)*100:.1f}%)")
    print(f"   Above target (>25): {above:,} ({above/len(valid_doses)*100:.1f}%)")

analyze_targets(crrt_analysis_df['crrt_dose_ml_kg_hr'], "At Initiation")
analyze_targets(dose_6h_summary['time_weighted_dose'], "First 6 Hours (Time-Weighted)")
analyze_targets(dose_24h_summary['time_weighted_dose'], "First 24 Hours (Time-Weighted)")

# ============================================================================
# Create Combined Analysis Dataset
# ============================================================================
print("\n" + "=" * 80)
print("Creating Combined Dose Dataset")
print("=" * 80)

# Merge all dose measurements
dose_comparison_df = crrt_analysis_df[['encounter_block', 'crrt_dose_ml_kg_hr']].copy()
dose_comparison_df = dose_comparison_df.rename(columns={'crrt_dose_ml_kg_hr': 'dose_at_initiation'})

dose_comparison_df = dose_comparison_df.merge(
    dose_6h_summary[['encounter_block', 'time_weighted_dose', 'n_measurements']],
    on='encounter_block',
    how='left',
    suffixes=('', '_6h')
).rename(columns={'time_weighted_dose': 'dose_6h_avg', 'n_measurements': 'n_measurements_6h'})

dose_comparison_df = dose_comparison_df.merge(
    dose_24h_summary[['encounter_block', 'time_weighted_dose', 'n_measurements']],
    on='encounter_block',
    how='left',
    suffixes=('', '_24h')
).rename(columns={'time_weighted_dose': 'dose_24h_avg', 'n_measurements': 'n_measurements_24h'})

print(f"\n   Combined dataset created:")
print(f"     Total encounters: {len(dose_comparison_df):,}")
print(f"     Has initiation dose: {dose_comparison_df['dose_at_initiation'].notna().sum():,}")
print(f"     Has 6h avg dose: {dose_comparison_df['dose_6h_avg'].notna().sum():,}")
print(f"     Has 24h avg dose: {dose_comparison_df['dose_24h_avg'].notna().sum():,}")

# Check correlation between time windows
print(f"\n   Correlation between dose measurements:")
print(dose_comparison_df[['dose_at_initiation', 'dose_6h_avg', 'dose_24h_avg']].corr())

print("\n✅ Time-weighted dose analysis completed!")
print(f"   Datasets created: 'dose_6h_summary', 'dose_24h_summary', 'dose_comparison_df'")



Step 10: Time-Weighted Average CRRT Dose Analysis

   CRRT records by time window:
     At initiation (t=0): 2,247 records
     Within 6 hours: 13,909 records
     Within 24 hours: 44,231 records

   Calculating doses for time windows...

   Calculating time-weighted averages...

CRRT Dose Comparison: Initiation vs 6h vs 24h

📍 At Initiation (t=0):
   Doses available: 776/2,247 (34.5%)
   Mean ± SD: 25.2 ± 53.1 mL/kg/hr
   Median [IQR]: 19.1 [9.0-30.3]

⏰ First 6 Hours:
   Doses available: 1,993/2,247 (88.7%)
   Mean ± SD: 53.8 ± 342.8 mL/kg/hr
   Median [IQR]: 32.1 [26.5-43.4]
   Avg measurements per encounter: 5.0

📅 First 24 Hours:
   Doses available: 2,074/2,247 (92.3%)
   Mean ± SD: 52.8 ± 166.4 mL/kg/hr
   Median [IQR]: 33.9 [27.3-47.4]
   Avg measurements per encounter: 17.2

Clinical Target Analysis (20-25 mL/kg/hr)

At Initiation:
   Below target (<20): 409 (52.7%)
   Within target (20-25): 76 (9.8%)
   Above target (>25): 291 (37.5%)

First 6 Hours (Time-Weighted):
   Below 

# Weight and DFR window availability

## Weight data availability 

In [45]:
print("=" * 80)
print("WEIGHT AVAILABILITY ANALYSIS")
print("=" * 80)

# Get all encounters from cohort
cohort_encounters = set(cohort_df['encounter_block'].unique())
print(f"\nTotal encounters in cohort: {len(cohort_encounters):,}")

# Check weight availability in wide_df (entire hospitalization)
print("\n1. WEIGHT AVAILABILITY ACROSS ENTIRE HOSPITALIZATION")
print("-" * 80)

# Get all weight records
all_weight_data = wide_df[['encounter_block', 'recorded_dttm', 'weight_kg']].copy()
all_weight_data = all_weight_data[all_weight_data['weight_kg'].notna()]

# Count encounters with ANY weight recorded
encounters_with_any_weight = set(all_weight_data['encounter_block'].unique())
encounters_without_any_weight = cohort_encounters - encounters_with_any_weight

print(f"Encounters with ANY weight recorded: {len(encounters_with_any_weight):,} ({len(encounters_with_any_weight)/len(cohort_encounters)*100:.1f}%)")
print(f"Encounters with NO weight at all: {len(encounters_without_any_weight):,} ({len(encounters_without_any_weight)/len(cohort_encounters)*100:.1f}%)")

# Check weight counts per encounter
weight_counts = all_weight_data.groupby('encounter_block').size()
print(f"\nWeight records per encounter:")
print(f"  Mean: {weight_counts.mean():.1f}")
print(f"  Median: {weight_counts.median():.0f}")
print(f"  Min-Max: {weight_counts.min():.0f} - {weight_counts.max():.0f}")

# Distribution of weight record counts
print(f"\nDistribution:")
print(f"  1 weight: {(weight_counts == 1).sum():,}")
print(f"  2-5 weights: {((weight_counts >= 2) & (weight_counts <= 5)).sum():,}")
print(f"  6-10 weights: {((weight_counts >= 6) & (weight_counts <= 10)).sum():,}")
print(f"  >10 weights: {(weight_counts > 10).sum():,}")

# 2. WEIGHT BEFORE CRRT INITIATION
print("\n2. WEIGHT AVAILABILITY BEFORE CRRT INITIATION")
print("-" * 80)

# Merge weight data with index times
weight_with_index = all_weight_data.merge(
    index_times[['encounter_block', 'index_time']],
    on='encounter_block',
    how='inner'
)

# Filter to weights BEFORE or AT index time
weight_before_crrt = weight_with_index[weight_with_index['recorded_dttm'] <= weight_with_index['index_time']]

encounters_with_weight_before = set(weight_before_crrt['encounter_block'].unique())
encounters_without_weight_before = cohort_encounters - encounters_with_weight_before

print(f"Encounters with weight BEFORE CRRT: {len(encounters_with_weight_before):,} ({len(encounters_with_weight_before)/len(cohort_encounters)*100:.1f}%)")
print(f"Encounters WITHOUT weight before CRRT: {len(encounters_without_weight_before):,} ({len(encounters_without_weight_before)/len(cohort_encounters)*100:.1f}%)")

# 3. DETAILED BREAKDOWN
print("\n3. DETAILED BREAKDOWN")
print("-" * 80)

# Category 1: No weight at all
cat1 = encounters_without_any_weight
print(f"Category 1 - NO weight recorded (entire hospitalization): {len(cat1):,}")

# Category 2: Has weight but ONLY after CRRT initiation
cat2 = encounters_with_any_weight - encounters_with_weight_before
print(f"Category 2 - Has weight but ONLY AFTER CRRT start: {len(cat2):,}")

# Category 3: Has weight before CRRT
cat3 = encounters_with_weight_before
print(f"Category 3 - Has weight BEFORE CRRT start: {len(cat3):,}")

print(f"\nVerification: {len(cat1)} + {len(cat2)} + {len(cat3)} = {len(cat1) + len(cat2) + len(cat3)} (should equal {len(cohort_encounters)})")

# 4. TIMING ANALYSIS FOR CATEGORY 2
if len(cat2) > 0:
    print("\n4. TIMING ANALYSIS - Weights recorded AFTER CRRT start")
    print("-" * 80)
    
    # For category 2 encounters, find when first weight appears
    cat2_weights = all_weight_data[all_weight_data['encounter_block'].isin(cat2)].merge(
        index_times[['encounter_block', 'index_time']],
        on='encounter_block'
    )
    
    cat2_weights['time_after_crrt_hours'] = (cat2_weights['recorded_dttm'] - cat2_weights['index_time']).dt.total_seconds() / 3600
    
    first_weight_after = cat2_weights.groupby('encounter_block')['time_after_crrt_hours'].min()
    
    print(f"First weight appears (hours after CRRT start):")
    print(f"  Mean: {first_weight_after.mean():.1f} hours")
    print(f"  Median: {first_weight_after.median():.1f} hours")
    print(f"  Range: {first_weight_after.min():.1f} - {first_weight_after.max():.1f} hours")
    
    print(f"\nDistribution:")
    print(f"  Within 6 hours: {(first_weight_after <= 6).sum():,}")
    print(f"  6-24 hours: {((first_weight_after > 6) & (first_weight_after <= 24)).sum():,}")
    print(f"  >24 hours: {(first_weight_after > 24).sum():,}")


print("\n" + "=" * 80)

WEIGHT AVAILABILITY ANALYSIS

Total encounters in cohort: 2,247

1. WEIGHT AVAILABILITY ACROSS ENTIRE HOSPITALIZATION
--------------------------------------------------------------------------------
Encounters with ANY weight recorded: 2,242 (99.8%)
Encounters with NO weight at all: 5 (0.2%)

Weight records per encounter:
  Mean: 20.1
  Median: 8
  Min-Max: 1 - 1459

Distribution:
  1 weight: 120
  2-5 weights: 744
  6-10 weights: 406
  >10 weights: 972

2. WEIGHT AVAILABILITY BEFORE CRRT INITIATION
--------------------------------------------------------------------------------
Encounters with weight BEFORE CRRT: 2,190 (97.5%)
Encounters WITHOUT weight before CRRT: 57 (2.5%)

3. DETAILED BREAKDOWN
--------------------------------------------------------------------------------
Category 1 - NO weight recorded (entire hospitalization): 5
Category 2 - Has weight but ONLY AFTER CRRT start: 52
Category 3 - Has weight BEFORE CRRT start: 2,190

Verification: 5 + 52 + 2190 = 2247 (should equa

## DFR availability

In [46]:
print("=" * 80)
print("DOSE CALCULATION AVAILABILITY ANALYSIS")
print("Examining when CRRT dose can be calculated: At initiation, within 6h, or within 24h")
print("=" * 80)

# Get all CRRT records for cohort encounters
cohort_encounters = set(cohort_df['encounter_block'].unique())
print(f"\nTotal encounters in cohort: {len(cohort_encounters):,}")

# Merge CRRT data with index times
crrt_with_timing = crrt_cohort.merge(
    index_times[['encounter_block', 'index_time']],
    on='encounter_block',
    how='inner'
)

# Calculate time from CRRT initiation
crrt_with_timing['hours_from_initiation'] = (
    crrt_with_timing['recorded_dttm'] - crrt_with_timing['index_time']
).dt.total_seconds() / 3600

# Define time windows
print("\n" + "=" * 80)
print("ANALYSIS BY TIME WINDOW")
print("=" * 80)

time_windows = [
    ('At Initiation (exact match)', 0, 0),
    ('Within 1 hour', 0, 1),
    ('Within 6 hours', 0, 6),
    ('Within 24 hours', 0, 24),
]

results = {}

for window_name, start_hr, end_hr in time_windows:
    print(f"\n{window_name}:")
    print("-" * 80)
    
    # Filter CRRT records to time window
    if start_hr == end_hr == 0:
        # Exact match at initiation
        window_crrt = crrt_with_timing[crrt_with_timing['hours_from_initiation'] == 0].copy()
    else:
        window_crrt = crrt_with_timing[
            (crrt_with_timing['hours_from_initiation'] >= start_hr) & 
            (crrt_with_timing['hours_from_initiation'] <= end_hr)
        ].copy()
    
    print(f"  Total CRRT records in window: {len(window_crrt):,}")
    print(f"  Unique encounters with CRRT data: {window_crrt['encounter_block'].nunique():,}")
    
    # Check for flow rates
    window_crrt['has_dfr'] = window_crrt['dialysate_flow_rate'] > 0
    window_crrt['has_pre_rfr'] = window_crrt['pre_filter_replacement_fluid_rate'] > 0
    window_crrt['has_post_rfr'] = window_crrt['post_filter_replacement_fluid_rate'] > 0
    window_crrt['has_any_flow'] = (
        window_crrt['has_dfr'] | 
        window_crrt['has_pre_rfr'] | 
        window_crrt['has_post_rfr']
    )
    
    # Count encounters with flow rates
    encounters_with_flows = window_crrt[window_crrt['has_any_flow']]['encounter_block'].unique()
    print(f"  Encounters with ANY flow rate: {len(encounters_with_flows):,} ({len(encounters_with_flows)/len(cohort_encounters)*100:.1f}%)")
    
    # Get first record with flows for each encounter
    first_with_flows = (window_crrt[window_crrt['has_any_flow']]
                       .sort_values(['encounter_block', 'recorded_dttm'])
                       .groupby('encounter_block')
                       .first()
                       .reset_index())
    
    # Merge with weights (using closest weight before or at that time)
    combined_temp = first_with_flows.merge(
        index_times[['encounter_block', 'index_time']], 
        on='encounter_block'
    )
    
    # Get weights available at that time
    weight_at_time = all_weight_data.merge(
        combined_temp[['encounter_block', 'recorded_dttm']], 
        on='encounter_block'
    )
    weight_at_time = weight_at_time[
        weight_at_time['recorded_dttm_x'] <= weight_at_time['recorded_dttm_y']
    ]
    
    # Get most recent weight for each encounter
    weights_available = (weight_at_time
                        .sort_values(['encounter_block', 'recorded_dttm_x'])
                        .groupby('encounter_block')
                        .last()
                        .reset_index())
    
    # Merge flows with weights
    dose_ready = first_with_flows.merge(
        weights_available[['encounter_block', 'weight_kg']],
        on='encounter_block',
        how='inner'
    )
    
    # Calculate dose
    dose_ready = dose_ready[dose_ready['crrt_mode_category'].notna()].copy()
    dose_ready['crrt_mode_category'] = dose_ready['crrt_mode_category'].str.lower()
    
    # Calculate total flow
    dose_ready[flow_cols] = dose_ready[flow_cols].fillna(0)
    
    conditions = [
        dose_ready['crrt_mode_category'] == 'cvvhd',
        dose_ready['crrt_mode_category'] == 'cvvh',
        dose_ready['crrt_mode_category'] == 'cvvhdf'
    ]
    
    choices = [
        dose_ready['dialysate_flow_rate'],
        dose_ready['pre_filter_replacement_fluid_rate'] + dose_ready['post_filter_replacement_fluid_rate'],
        dose_ready['dialysate_flow_rate'] + dose_ready['pre_filter_replacement_fluid_rate'] + dose_ready['post_filter_replacement_fluid_rate']
    ]
    
    dose_ready['total_flow'] = np.select(conditions, choices, default=np.nan)
    
    dose_ready['dose_calculable'] = (
        (dose_ready['weight_kg'] > 0) & 
        (dose_ready['total_flow'] > 0)
    )
    
    n_dose_calculable = dose_ready['dose_calculable'].sum()
    
    print(f"  Encounters with BOTH weight AND flow: {n_dose_calculable:,} ({n_dose_calculable/len(cohort_encounters)*100:.1f}%)")
    
    # Store results
    results[window_name] = {
        'encounters_with_crrt': window_crrt['encounter_block'].nunique(),
        'encounters_with_flows': len(encounters_with_flows),
        'encounters_dose_calculable': n_dose_calculable,
        'percentage': n_dose_calculable/len(cohort_encounters)*100
    }

# Summary comparison
print("\n" + "=" * 80)
print("SUMMARY COMPARISON")
print("=" * 80)

print(f"\n{'Time Window':<30} {'Has CRRT':<12} {'Has Flows':<12} {'Dose Ready':<12} {'%':<8}")
print("-" * 80)

for window_name, stats in results.items():
    print(f"{window_name:<30} {stats['encounters_with_crrt']:<12,} {stats['encounters_with_flows']:<12,} {stats['encounters_dose_calculable']:<12,} {stats['percentage']:<7.1f}%")

print("\n" + "=" * 80)

DOSE CALCULATION AVAILABILITY ANALYSIS
Examining when CRRT dose can be calculated: At initiation, within 6h, or within 24h

Total encounters in cohort: 2,247

ANALYSIS BY TIME WINDOW

At Initiation (exact match):
--------------------------------------------------------------------------------
  Total CRRT records in window: 2,247
  Unique encounters with CRRT data: 2,247
  Encounters with ANY flow rate: 799 (35.6%)
  Encounters with BOTH weight AND flow: 776 (34.5%)

Within 1 hour:
--------------------------------------------------------------------------------
  Total CRRT records in window: 4,276
  Unique encounters with CRRT data: 2,247
  Encounters with ANY flow rate: 1,855 (82.6%)
  Encounters with BOTH weight AND flow: 1,806 (80.4%)

Within 6 hours:
--------------------------------------------------------------------------------
  Total CRRT records in window: 13,909
  Unique encounters with CRRT data: 2,247
  Encounters with ANY flow rate: 2,048 (91.1%)
  Encounters with BOTH we

In [47]:
crrt_without_flows = crrt_df[~crrt_df['hospitalization_id'].isin(first_with_flows['hospitalization_id'])]

In [48]:
# ============================================================================
# Identify Encounters Missing Dose Within First 6 Hours
# ============================================================================
print("\n" + "=" * 80)
print("Analysis: Missing CRRT Dose Within First 6 Hours")
print("=" * 80)

# Get all CRRT records within 6 hours of initiation with index time
crrt_6h = crrt_with_index[
    (crrt_with_index['hours_from_index'] >= 0) & 
    (crrt_with_index['hours_from_index'] <= 6)
].copy()

# Calculate dose for 6h window (reusing the calculation function from earlier)
crrt_6h_doses = calculate_crrt_dose(crrt_6h, closest_weights)

# Identify which encounters have at least one valid dose within 6h
encounters_with_dose_6h = crrt_6h_doses[
    crrt_6h_doses['crrt_dose_ml_kg_hr'].notna()
]['encounter_block'].unique()

# All encounters in cohort
all_encounters = cohort_df['encounter_block'].unique()

# Encounters WITHOUT any dose within 6h
missing_dose_6h = set(all_encounters) - set(encounters_with_dose_6h)

print(f"\n📊 Overall Summary:")
print(f"   Total CRRT encounters: {len(all_encounters):,}")
print(f"   Have dose within 6h: {len(encounters_with_dose_6h):,} ({len(encounters_with_dose_6h)/len(all_encounters)*100:.1f}%)")
print(f"   Missing dose within 6h: {len(missing_dose_6h):,} ({len(missing_dose_6h)/len(all_encounters)*100:.1f}%)")

# ============================================================================
# Detailed Breakdown of Why Dose is Missing
# ============================================================================
print("\n" + "-" * 80)
print("Breakdown: Why is Dose Missing Within First 6 Hours?")
print("-" * 80)

# Create dataframe of missing encounters for analysis
missing_encounters_df = pd.DataFrame({'encounter_block': list(missing_dose_6h)})

# Get their CRRT records within 6h
missing_crrt_6h = crrt_6h[crrt_6h['encounter_block'].isin(missing_dose_6h)].copy()

print(f"\nTotal CRRT records within 6h for missing encounters: {len(missing_crrt_6h):,}")

# Reason 1: No CRRT records at all within 6h
no_records_6h = missing_encounters_df[
    ~missing_encounters_df['encounter_block'].isin(missing_crrt_6h['encounter_block'])
]
print(f"\n1️⃣ No CRRT records within 6h: {len(no_records_6h):,} encounters")

if len(missing_crrt_6h) > 0:
    # Merge with weights
    missing_crrt_6h = missing_crrt_6h.merge(
        closest_weights[['encounter_block', 'weight_kg']],
        on='encounter_block',
        how='left'
    )
    
    # Reason 2: Missing weight
    encounters_no_weight = missing_crrt_6h[
        missing_crrt_6h['weight_kg'].isna()
    ]['encounter_block'].unique()
    print(f"\n2️⃣ Missing weight: {len(encounters_no_weight):,} encounters")
    
    # For those WITH weight, check flow rates
    with_weight = missing_crrt_6h[missing_crrt_6h['weight_kg'].notna()].copy()
    
    if len(with_weight) > 0:
        # Check individual flow components
        flow_cols = ['dialysate_flow_rate', 'pre_filter_replacement_fluid_rate', 
                     'post_filter_replacement_fluid_rate']
        
        print(f"\n3️⃣ Have weight but missing/zero flow rates: {len(with_weight['encounter_block'].unique()):,} encounters")
        print(f"   Total records with weight: {len(with_weight):,}")
        
        # Detailed flow rate analysis
        print(f"\n   Flow Rate Availability (among records with weight):")
        for col in flow_cols:
            available = (with_weight[col] > 0).sum()
            print(f"     {col}: {available:,}/{len(with_weight):,} records ({available/len(with_weight)*100:.1f}%)")
        
        # Check if ANY flow rate is present
        with_weight['has_any_flow'] = (
            (with_weight['dialysate_flow_rate'] > 0) |
            (with_weight['pre_filter_replacement_fluid_rate'] > 0) |
            (with_weight['post_filter_replacement_fluid_rate'] > 0)
        )
        
        no_flow = with_weight[~with_weight['has_any_flow']]
        print(f"\n   Records with NO flow rates at all: {len(no_flow):,} ({len(no_flow)/len(with_weight)*100:.1f}%)")
        print(f"   Encounters affected: {no_flow['encounter_block'].nunique():,}")
        
        # Reason 4: Wrong/missing CRRT mode
        with_weight['crrt_mode_category'] = with_weight['crrt_mode_category'].str.lower()
        unknown_mode = with_weight[
            ~with_weight['crrt_mode_category'].isin(['cvvhd', 'cvvh', 'cvvhdf'])
        ]
        
        if len(unknown_mode) > 0:
            print(f"\n4️⃣ Unknown/missing CRRT mode: {len(unknown_mode):,} records")
            print(f"   Encounters affected: {unknown_mode['encounter_block'].nunique():,}")
            print(f"   Modes found: {unknown_mode['crrt_mode_category'].value_counts().to_dict()}")

# ============================================================================
# Create Dataset of Missing Encounters
# ============================================================================
print("\n" + "-" * 80)
print("Creating Dataset of Missing Encounters")
print("-" * 80)

# Merge with cohort data to get patient characteristics
missing_encounters_detailed = missing_encounters_df.merge(
    cohort_df,
    on='encounter_block',
    how='left'
)

# Add reason codes
missing_encounters_detailed['has_crrt_records_6h'] = missing_encounters_detailed['encounter_block'].isin(
    missing_crrt_6h['encounter_block']
)

missing_encounters_detailed['has_weight'] = missing_encounters_detailed['encounter_block'].isin(
    missing_crrt_6h[missing_crrt_6h['weight_kg'].notna()]['encounter_block']
)

if len(with_weight) > 0:
    missing_encounters_detailed['has_any_flow_6h'] = missing_encounters_detailed['encounter_block'].isin(
        with_weight[with_weight['has_any_flow']]['encounter_block']
    )
else:
    missing_encounters_detailed['has_any_flow_6h'] = False

# Classify primary reason for missing dose
def classify_missing_reason(row):
    if not row['has_crrt_records_6h']:
        return 'No CRRT records in 6h'
    elif not row['has_weight']:
        return 'Missing weight'
    elif not row['has_any_flow_6h']:
        return 'No flow rates recorded'
    else:
        return 'Other (mode/calculation issue)'

missing_encounters_detailed['missing_reason'] = missing_encounters_detailed.apply(
    classify_missing_reason, axis=1
)

print(f"\nPrimary Reasons for Missing Dose:")
reason_counts = missing_encounters_detailed['missing_reason'].value_counts()
for reason, count in reason_counts.items():
    print(f"   {reason}: {count:,} ({count/len(missing_encounters_detailed)*100:.1f}%)")

# ============================================================================
# Compare Characteristics: Missing vs Available Dose
# ============================================================================
print("\n" + "-" * 80)
print("Comparing Encounters: Missing vs Available Dose Within 6h")
print("-" * 80)

# Create comparison groups
encounters_available_df = cohort_df[
    cohort_df['encounter_block'].isin(encounters_with_dose_6h)
].copy()
encounters_available_df['dose_available_6h'] = True

missing_encounters_detailed['dose_available_6h'] = False

comparison_df = pd.concat([
    encounters_available_df[['encounter_block', 'dose_available_6h']],
    missing_encounters_detailed[['encounter_block', 'dose_available_6h']]
])

# Merge with any other characteristics you have
# For example, if you have SOFA scores:
if 'sofa_df' in dir() or 'sofa_df' in locals():
    comparison_df = comparison_df.merge(
        sofa_df[['encounter_block', 'sofa_total']],
        on='encounter_block',
        how='left'
    )
    
    print(f"\nSOFA Score Comparison:")
    for has_dose in [True, False]:
        subset = comparison_df[comparison_df['dose_available_6h'] == has_dose]
        sofa_valid = subset['sofa_total'].dropna()
        label = "Dose Available" if has_dose else "Dose Missing"
        
        if len(sofa_valid) > 0:
            print(f"   {label}:")
            print(f"     N = {len(sofa_valid):,}")
            print(f"     Mean SOFA: {sofa_valid.mean():.1f} ± {sofa_valid.std():.1f}")
            print(f"     Median SOFA: {sofa_valid.median():.1f}")

print("\n" + "=" * 80)
print("✅ Analysis Complete")
print(f"   Dataset created: 'missing_encounters_detailed' ({len(missing_encounters_detailed):,} encounters)")
print("=" * 80)

# Export for review
print("\nSample of missing encounters:")
print(missing_encounters_detailed[['encounter_block', 'hospitalization_id', 
                                   'missing_reason', 'has_crrt_records_6h', 
                                   'has_weight', 'has_any_flow_6h']].head(10))


Analysis: Missing CRRT Dose Within First 6 Hours

📊 Overall Summary:
   Total CRRT encounters: 2,247
   Have dose within 6h: 1,993 (88.7%)
   Missing dose within 6h: 254 (11.3%)

--------------------------------------------------------------------------------
Breakdown: Why is Dose Missing Within First 6 Hours?
--------------------------------------------------------------------------------

Total CRRT records within 6h for missing encounters: 1,001

1️⃣ No CRRT records within 6h: 0 encounters

2️⃣ Missing weight: 57 encounters

3️⃣ Have weight but missing/zero flow rates: 197 encounters
   Total records with weight: 633

   Flow Rate Availability (among records with weight):
     dialysate_flow_rate: 0/633 records (0.0%)
     pre_filter_replacement_fluid_rate: 0/633 records (0.0%)
     post_filter_replacement_fluid_rate: 0/633 records (0.0%)

   Records with NO flow rates at all: 0 (0.0%)
   Encounters affected: 0

4️⃣ Unknown/missing CRRT mode: 148 records
   Encounters affected: 30

In [49]:
# ============================================================================
# Examine CRRT Data for Encounters Missing Dose Within 6 Hours
# ============================================================================
print("\n" + "=" * 80)
print("Detailed CRRT Data Examination: Missing Dose Encounters")
print("=" * 80)

# Get list of encounters missing dose
missing_encounter_list = missing_encounters_detailed['encounter_block'].unique()

print(f"\nTotal encounters to examine: {len(missing_encounter_list):,}")

# ============================================================================
# Filter CRRT data for these encounters
# ============================================================================

# All CRRT records for missing encounters
missing_crrt_full = clif.crrt_therapy.df[
    clif.crrt_therapy.df['encounter_block'].isin(missing_encounter_list)
].copy()

print(f"\n📊 CRRT Records Overview:")
print(f"   Total CRRT records: {len(missing_crrt_full):,}")
print(f"   Unique encounters: {missing_crrt_full['encounter_block'].nunique():,}")
print(f"   Unique hospitalizations: {missing_crrt_full['hospitalization_id'].nunique():,}")
print(f"   Date range: {missing_crrt_full['recorded_dttm'].min()} to {missing_crrt_full['recorded_dttm'].max()}")

# Records per encounter
records_per_encounter = missing_crrt_full.groupby('encounter_block').size()
print(f"\n   Records per encounter:")
print(f"     Mean: {records_per_encounter.mean():.1f}")
print(f"     Median: {records_per_encounter.median():.0f}")
print(f"     Range: {records_per_encounter.min()} - {records_per_encounter.max()}")

# ============================================================================
# Focus on first 6 hours only
# ============================================================================

# Add index time to filter for first 6 hours
missing_crrt_full = missing_crrt_full.merge(
    index_times,
    on='encounter_block',
    how='left'
)

missing_crrt_full['hours_from_index'] = (
    missing_crrt_full['recorded_dttm'] - missing_crrt_full['index_time']
).dt.total_seconds() / 3600

# Filter for first 6 hours
missing_crrt_6h_only = missing_crrt_full[
    (missing_crrt_full['hours_from_index'] >= 0) & 
    (missing_crrt_full['hours_from_index'] <= 6)
].copy()

print(f"\n📍 First 6 Hours Only:")
print(f"   CRRT records: {len(missing_crrt_6h_only):,}")
print(f"   Encounters with any records: {missing_crrt_6h_only['encounter_block'].nunique():,}")
print(f"   Encounters with NO records in 6h: {len(missing_encounter_list) - missing_crrt_6h_only['encounter_block'].nunique():,}")

# ============================================================================
# Analyze Data Completeness
# ============================================================================
print("\n" + "-" * 80)
print("Data Completeness Analysis (First 6 Hours)")
print("-" * 80)

key_columns = [
    'crrt_mode_category',
    'dialysate_flow_rate',
    'pre_filter_replacement_fluid_rate',
    'post_filter_replacement_fluid_rate',
    'ultrafiltration_out',
    'blood_flow_rate'
]

print(f"\n{'Column':<45} {'Records':<12} {'% Complete':<12}")
print("-" * 69)

for col in key_columns:
    if col in missing_crrt_6h_only.columns:
        available = missing_crrt_6h_only[col].notna().sum()
        non_zero = (missing_crrt_6h_only[col] > 0).sum() if missing_crrt_6h_only[col].dtype in ['float64', 'int64', 'Float64', 'Int64'] else available
        total = len(missing_crrt_6h_only)
        
        print(f"{col:<45} {available:>6,} ({available/total*100:>5.1f}%)   Non-zero: {non_zero:>6,} ({non_zero/total*100:>5.1f}%)")
    else:
        print(f"{col:<45} Column not found")

# ============================================================================
# CRRT Mode Distribution
# ============================================================================
print("\n" + "-" * 80)
print("CRRT Mode Distribution")
print("-" * 80)

if len(missing_crrt_6h_only) > 0:
    mode_counts = missing_crrt_6h_only['crrt_mode_category'].value_counts(dropna=False)
    print(f"\n{'Mode':<30} {'Count':<10} {'%'}")
    print("-" * 45)
    for mode, count in mode_counts.items():
        mode_str = str(mode) if pd.notna(mode) else 'Missing/NaN'
        print(f"{mode_str:<30} {count:<10,} {count/len(missing_crrt_6h_only)*100:>5.1f}%")

# ============================================================================
# Flow Rate Statistics
# ============================================================================
print("\n" + "-" * 80)
print("Flow Rate Statistics (First 6 Hours)")
print("-" * 80)

flow_cols = ['dialysate_flow_rate', 'pre_filter_replacement_fluid_rate', 
             'post_filter_replacement_fluid_rate', 'ultrafiltration_out']

for col in flow_cols:
    if col in missing_crrt_6h_only.columns:
        valid = missing_crrt_6h_only[col].dropna()
        non_zero = valid[valid > 0]
        
        print(f"\n{col}:")
        print(f"   Non-null: {len(valid):,}/{len(missing_crrt_6h_only):,} ({len(valid)/len(missing_crrt_6h_only)*100:.1f}%)")
        print(f"   Non-zero: {len(non_zero):,} ({len(non_zero)/len(missing_crrt_6h_only)*100:.1f}%)")
        
        if len(non_zero) > 0:
            print(f"   Mean: {non_zero.mean():.1f} mL/hr")
            print(f"   Median: {non_zero.median():.1f} mL/hr")
            print(f"   Range: {non_zero.min():.1f} - {non_zero.max():.1f} mL/hr")

# ============================================================================
# Sample Records for Manual Review
# ============================================================================
print("\n" + "-" * 80)
print("Sample Records for Manual Review")
print("-" * 80)

# Select columns for review
review_cols = [
    'encounter_block', 'hospitalization_id', 'recorded_dttm', 'hours_from_index',
    'crrt_mode_category', 'dialysate_flow_rate', 
    'pre_filter_replacement_fluid_rate', 'post_filter_replacement_fluid_rate',
    'ultrafiltration_out', 'blood_flow_rate'
]

# Filter to columns that exist
review_cols_exist = [col for col in review_cols if col in missing_crrt_6h_only.columns]

# Get sample from first 10 encounters
sample_encounters = missing_crrt_6h_only['encounter_block'].unique()[:10]
sample_records = missing_crrt_6h_only[
    missing_crrt_6h_only['encounter_block'].isin(sample_encounters)
][review_cols_exist].sort_values(['encounter_block', 'hours_from_index'])

print(f"\nShowing records from first 10 encounters:")
print(sample_records.to_string())

# ============================================================================
# Identify Specific Problematic Patterns
# ============================================================================
print("\n" + "-" * 80)
print("Problematic Patterns Identified")
print("-" * 80)

if len(missing_crrt_6h_only) > 0:
    # Pattern 1: Records with no flow rates at all
    no_flow_mask = (
        (missing_crrt_6h_only['dialysate_flow_rate'].fillna(0) == 0) &
        (missing_crrt_6h_only['pre_filter_replacement_fluid_rate'].fillna(0) == 0) &
        (missing_crrt_6h_only['post_filter_replacement_fluid_rate'].fillna(0) == 0)
    )
    no_flow_encounters = missing_crrt_6h_only[no_flow_mask]['encounter_block'].nunique()
    print(f"\n1️⃣ Encounters with records but NO flow rates: {no_flow_encounters:,}")
    print(f"   Total records affected: {no_flow_mask.sum():,}")
    
    # Pattern 2: Missing CRRT mode
    missing_mode = missing_crrt_6h_only['crrt_mode_category'].isna()
    missing_mode_encounters = missing_crrt_6h_only[missing_mode]['encounter_block'].nunique()
    print(f"\n2️⃣ Encounters with missing CRRT mode: {missing_mode_encounters:,}")
    print(f"   Total records affected: {missing_mode.sum():,}")
    
    # Pattern 3: Unrecognized modes
    valid_modes = ['cvvhd', 'cvvh', 'cvvhdf', 'scuf']
    if 'crrt_mode_category' in missing_crrt_6h_only.columns:
        invalid_mode_mask = (
            missing_crrt_6h_only['crrt_mode_category'].notna() &
            ~missing_crrt_6h_only['crrt_mode_category'].str.lower().isin(valid_modes)
        )
        if invalid_mode_mask.sum() > 0:
            invalid_modes = missing_crrt_6h_only[invalid_mode_mask]['crrt_mode_category'].unique()
            print(f"\n3️⃣ Unrecognized CRRT modes found: {invalid_modes}")
            print(f"   Total records affected: {invalid_mode_mask.sum():,}")

# ============================================================================
# Export for Detailed Review
# ============================================================================
print("\n" + "=" * 80)
print("✅ Analysis Complete")
print("=" * 80)

print(f"\nDatasets created:")
print(f"   'missing_crrt_full' - All CRRT records for missing encounters ({len(missing_crrt_full):,} records)")
print(f"   'missing_crrt_6h_only' - First 6h only ({len(missing_crrt_6h_only):,} records)")
print(f"   'sample_records' - Sample for review ({len(sample_records):,} records)")

# Optionally save to CSV for external review
# missing_crrt_6h_only.to_csv('output/missing_dose_crrt_records_6h.csv', index=False)
# print("\n💾 Exported to: output/missing_dose_crrt_records_6h.csv")


Detailed CRRT Data Examination: Missing Dose Encounters

Total encounters to examine: 254

📊 CRRT Records Overview:
   Total CRRT records: 26,376
   Unique encounters: 254
   Unique hospitalizations: 254
   Date range: 2018-01-16 16:00:00-06:00 to 2024-12-27 04:00:00-06:00

   Records per encounter:
     Mean: 103.8
     Median: 36
     Range: 1 - 1944

📍 First 6 Hours Only:
   CRRT records: 1,001
   Encounters with any records: 254
   Encounters with NO records in 6h: 0

--------------------------------------------------------------------------------
Data Completeness Analysis (First 6 Hours)
--------------------------------------------------------------------------------

Column                                        Records      % Complete  
---------------------------------------------------------------------
crrt_mode_category                             1,001 (100.0%)   Non-zero:  1,001 (100.0%)
dialysate_flow_rate                              469 ( 46.9%)   Non-zero:    318 ( 3

In [50]:
197/2190

0.08995433789954338

In [51]:
# ============================================================================
# Add Weight Column to Missing CRRT Data
# ============================================================================
print("\n" + "=" * 80)
print("Adding Weight Data to Missing CRRT Records")
print("=" * 80)

# Merge the pre-CRRT weights (closest_weights) with missing_crrt_full
missing_crrt_full = missing_crrt_full.merge(
    closest_weights[['encounter_block', 'weight_kg']],
    on='encounter_block',
    how='left'
)

print(f"\n✅ Weight column added to 'missing_crrt_full'")
print(f"\n   Weight Availability:")
print(f"     Total records: {len(missing_crrt_full):,}")
print(f"     Records with weight: {missing_crrt_full['weight_kg'].notna().sum():,} ({missing_crrt_full['weight_kg'].notna().mean()*100:.1f}%)")
print(f"     Records missing weight: {missing_crrt_full['weight_kg'].isna().sum():,} ({missing_crrt_full['weight_kg'].isna().mean()*100:.1f}%)")

# By unique encounters
print(f"\n   By Unique Encounters:")
weight_by_encounter = missing_crrt_full.groupby('encounter_block')['weight_kg'].first()
print(f"     Total encounters: {len(weight_by_encounter):,}")
print(f"     Encounters with weight: {weight_by_encounter.notna().sum():,} ({weight_by_encounter.notna().mean()*100:.1f}%)")
print(f"     Encounters missing weight: {weight_by_encounter.isna().sum():,} ({weight_by_encounter.isna().mean()*100:.1f}%)")

# Weight statistics for those that have it
if missing_crrt_full['weight_kg'].notna().sum() > 0:
    weight_stats = missing_crrt_full['weight_kg'].describe()
    print(f"\n   Weight Statistics (kg):")
    print(f"     Mean ± SD: {weight_stats['mean']:.1f} ± {weight_stats['std']:.1f}")
    print(f"     Median [IQR]: {weight_stats['50%']:.1f} [{weight_stats['25%']:.1f}-{weight_stats['75%']:.1f}]")
    print(f"     Range: {weight_stats['min']:.1f} - {weight_stats['max']:.1f}")

# Also update missing_crrt_6h_only with weight
missing_crrt_6h_only = missing_crrt_6h_only.merge(
    closest_weights[['encounter_block', 'weight_kg']],
    on='encounter_block',
    how='left',
    suffixes=('_old', '')  # In case weight was already there
)

# Remove old weight column if it exists
if 'weight_kg_old' in missing_crrt_6h_only.columns:
    missing_crrt_6h_only = missing_crrt_6h_only.drop(columns=['weight_kg_old'])

print(f"\n✅ Weight column also added to 'missing_crrt_6h_only'")
print(f"     Records with weight: {missing_crrt_6h_only['weight_kg'].notna().sum():,} ({missing_crrt_6h_only['weight_kg'].notna().mean()*100:.1f}%)")

# ============================================================================
# Updated Sample with Weight Column
# ============================================================================
print("\n" + "-" * 80)
print("Sample Records with Weight (First 6 Hours)")
print("-" * 80)

# Updated review columns including weight
review_cols = [
    'encounter_block', 'hospitalization_id', 'recorded_dttm', 'hours_from_index',
    'weight_kg',  # <-- Added weight
    'crrt_mode_category', 'dialysate_flow_rate', 
    'pre_filter_replacement_fluid_rate', 'post_filter_replacement_fluid_rate',
    'ultrafiltration_out', 'blood_flow_rate'
]

# Filter to columns that exist
review_cols_exist = [col for col in review_cols if col in missing_crrt_6h_only.columns]

# Get sample from first 10 encounters
sample_encounters = missing_crrt_6h_only['encounter_block'].unique()[:10]
sample_records_with_weight = missing_crrt_6h_only[
    missing_crrt_6h_only['encounter_block'].isin(sample_encounters)
][review_cols_exist].sort_values(['encounter_block', 'hours_from_index'])

print(f"\nShowing records from first 10 encounters (with weight):")
print(sample_records_with_weight.to_string())

print("\n" + "=" * 80)


Adding Weight Data to Missing CRRT Records

✅ Weight column added to 'missing_crrt_full'

   Weight Availability:
     Total records: 26,376
     Records with weight: 16,717 (63.4%)
     Records missing weight: 9,659 (36.6%)

   By Unique Encounters:
     Total encounters: 254
     Encounters with weight: 197 (77.6%)
     Encounters missing weight: 57 (22.4%)

   Weight Statistics (kg):
     Mean ± SD: 96.8 ± 27.7
     Median [IQR]: 89.3 [79.6-108.1]
     Range: 45.0 - 210.0

✅ Weight column also added to 'missing_crrt_6h_only'
     Records with weight: 633 (63.2%)

--------------------------------------------------------------------------------
Sample Records with Weight (First 6 Hours)
--------------------------------------------------------------------------------

Showing records from first 10 encounters (with weight):
     encounter_block hospitalization_id             recorded_dttm  hours_from_index   weight_kg crrt_mode_category  dialysate_flow_rate  pre_filter_replacement_flui

## Labs availability

In [52]:
print("=" * 80)
print("Extracting Labs at CRRT Initiation")
print("=" * 80)

# Define labs we want to extract
labs_to_extract = ['ph_arterial', 'lactate', 'bicarbonate', 'potassium']

# Extract lab data with encounter_block
lab_data = wide_df[['encounter_block', 'recorded_dttm'] + labs_to_extract].copy()

print(f"\nTotal lab records in wide_df: {len(lab_data):,}")

# Merge with index times
labs_with_index = lab_data.merge(
    index_times,
    on='encounter_block',
    how='inner'
)

print(f"Lab records for cohort encounters: {len(labs_with_index):,}")

# Get labs at exact index time
labs_at_index = labs_with_index[
    labs_with_index['recorded_dttm'] == labs_with_index['index_time']
].copy()

print(f"\nLabs at exact index time: {len(labs_at_index):,}")

# Check availability
print(f"\nLab availability at index time:")
for lab in labs_to_extract:
    available = labs_at_index[lab].notna().sum()
    print(f"  {lab}: {available:,} ({available/len(index_times)*100:.1f}%)")

# Create final labs dataset
labs_at_initiation = labs_at_index[['encounter_block', 'index_time'] + labs_to_extract].copy()

print(f"\n✅ Labs at initiation extracted: {len(labs_at_initiation):,} encounters")

# If exact match gives too few results, use closest within window
if len(labs_at_initiation) < len(index_times) * 0.5:  # Less than 50%
    print("\n⚠️  Low coverage at exact time. Extracting closest labs within 6-hour window...")
    
    # Get labs within 6 hours before or after index time
    labs_with_index['time_diff_hours'] = abs(
        (labs_with_index['recorded_dttm'] - labs_with_index['index_time']).dt.total_seconds() / 3600
    )
    
    labs_near_index = labs_with_index[labs_with_index['time_diff_hours'] <= 6].copy()
    
    print(f"Lab records within 6-hour window: {len(labs_near_index):,}")
    
    # For each encounter and each lab, get the closest value
    closest_labs = []
    
    for encounter in index_times['encounter_block']:
        encounter_labs = labs_near_index[labs_near_index['encounter_block'] == encounter]
        
        if len(encounter_labs) == 0:
            continue
        
        lab_values = {'encounter_block': encounter}
        
        for lab in labs_to_extract:
            # Get records where this lab is not null
            lab_records = encounter_labs[encounter_labs[lab].notna()]
            
            if len(lab_records) > 0:
                # Get the closest one
                closest_idx = lab_records['time_diff_hours'].idxmin()
                lab_values[lab] = lab_records.loc[closest_idx, lab]
            else:
                lab_values[lab] = np.nan
        
        closest_labs.append(lab_values)
    
    labs_at_initiation = pd.DataFrame(closest_labs)
    
    # Merge back with index_time
    labs_at_initiation = labs_at_initiation.merge(
        index_times[['encounter_block', 'index_time']],
        on='encounter_block',
        how='left'
    )
    
    print(f"\nUpdated lab availability (within 6-hour window):")
    for lab in labs_to_extract:
        available = labs_at_initiation[lab].notna().sum()
        print(f"  {lab}: {available:,} ({available/len(index_times)*100:.1f}%)")

# Display summary
print("\n" + "=" * 80)
print("SUMMARY")
print("=" * 80)
print(f"Total encounters: {len(index_times):,}")
print(f"Encounters with lab data: {len(labs_at_initiation):,}")

print(f"\nLab Statistics (at/near CRRT initiation):")
for lab in labs_to_extract:
    lab_values = labs_at_initiation[lab].dropna()
    if len(lab_values) > 0:
        print(f"\n{lab}:")
        print(f"  Count: {len(lab_values):,}")
        print(f"  Mean ± SD: {lab_values.mean():.2f} ± {lab_values.std():.2f}")
        print(f"  Median [IQR]: {lab_values.median():.2f} [{lab_values.quantile(0.25):.2f}-{lab_values.quantile(0.75):.2f}]")
        print(f"  Range: {lab_values.min():.2f} - {lab_values.max():.2f}")

print("\n✅ Labs at initiation dataset created: 'labs_at_initiation'")
print(f"\nSample:")
print(labs_at_initiation.head())

Extracting Labs at CRRT Initiation

Total lab records in wide_df: 2,044,721
Lab records for cohort encounters: 2,044,721

Labs at exact index time: 1,270

Lab availability at index time:
  ph_arterial: 7 (0.3%)
  lactate: 13 (0.6%)
  bicarbonate: 13 (0.6%)
  potassium: 10 (0.4%)

✅ Labs at initiation extracted: 1,270 encounters

SUMMARY
Total encounters: 2,247
Encounters with lab data: 1,270

Lab Statistics (at/near CRRT initiation):

ph_arterial:
  Count: 7
  Mean ± SD: 7.21 ± 0.14
  Median [IQR]: 7.24 [7.11-7.32]
  Range: 7.00 - 7.35

lactate:
  Count: 13
  Mean ± SD: 9.22 ± 4.83
  Median [IQR]: 9.30 [5.60-13.00]
  Range: 2.20 - 16.50

bicarbonate:
  Count: 13
  Mean ± SD: 21.30 ± 9.03
  Median [IQR]: 21.00 [18.00-24.00]
  Range: 5.90 - 44.00

potassium:
  Count: 10
  Mean ± SD: 4.93 ± 1.08
  Median [IQR]: 4.70 [4.22-5.42]
  Range: 3.50 - 7.30

✅ Labs at initiation dataset created: 'labs_at_initiation'

Sample:
      encounter_block                index_time  ph_arterial  lactate  \


# SOFA Score calculation

In [53]:
print("Initializing ClifOrchestrator for SOFA computation...")
co = ClifOrchestrator(
    data_directory=config['tables_path'],
    filetype=config['file_type'],
    timezone=config['timezone']
)
print("✅ ClifOrchestrator initialized")

Initializing ClifOrchestrator for SOFA computation...
ClifOrchestrator initialized.
✅ ClifOrchestrator initialized


In [54]:
sofa_cohort_df = index_times.copy()
sofa_cohort_df['start_time'] = sofa_cohort_df['index_time']
sofa_cohort_df['end_time'] = sofa_cohort_df['index_time'] + pd.Timedelta(hours=6)

# Join with cohort_df to get hospitalization_id
sofa_cohort_df = sofa_cohort_df.merge(
    cohort_df[['encounter_block', 'hospitalization_id']],
    on='encounter_block',
    how='left'
)

# Keep only required columns
sofa_cohort_df = sofa_cohort_df[['hospitalization_id','encounter_block', 'start_time', 'end_time']]
sofa_cohort_ids = cohort_df['hospitalization_id'].astype(str).unique().tolist()

In [55]:
# Load required tables for SOFA computation with cohort filtering
print("Loading required tables for SOFA computation...")
print("SOFA requires: Labs (creatinine, platelet_count, po2_arterial, bilirubin_total)")
print("               Vitals (map, spo2)")
print("               Assessments (gcs_total)")
print("               Medications (norepinephrine, epinephrine, dopamine, dobutamine)")
print("               Respiratory (device_category, fio2_set)")

# Define columns to load for each table (optimize memory usage)
sofa_columns = {
    'labs': ['hospitalization_id', 'lab_result_dttm', 'lab_category', 'lab_value', 'lab_value_numeric'],
    'vitals': ['hospitalization_id', 'recorded_dttm', 'vital_category', 'vital_value'],
    'patient_assessments': ['hospitalization_id', 'recorded_dttm', 'assessment_category', 'numerical_value'],
    'medication_admin_continuous': None,  # Load all columns
    'respiratory_support': None  # Load all columns
}

sofa_tables = ['labs', 'vitals', 'patient_assessments', 'medication_admin_continuous', 'respiratory_support']

for table_name in sofa_tables:
    table_cols = sofa_columns.get(table_name)
    print(f"Loading {table_name} with {len(table_cols) if table_cols else 'all'} columns and {len(sofa_cohort_ids)} hospitalization filters...")
    co.load_table(
        table_name,
        filters={'hospitalization_id': sofa_cohort_ids},
        columns=table_cols
    )

co.encounter_mapping = encounter_mapping[encounter_mapping['hospitalization_id'].isin(cohort_df['hospitalization_id'])]
print("✅ All required tables loaded for SOFA computation")

Loading required tables for SOFA computation...
SOFA requires: Labs (creatinine, platelet_count, po2_arterial, bilirubin_total)
               Vitals (map, spo2)
               Assessments (gcs_total)
               Medications (norepinephrine, epinephrine, dopamine, dobutamine)
               Respiratory (device_category, fio2_set)
Loading labs with 5 columns and 2247 hospitalization filters...
Loading vitals with 4 columns and 2247 hospitalization filters...
Loading patient_assessments with 4 columns and 2247 hospitalization filters...
Loading medication_admin_continuous with all columns and 2247 hospitalization filters...
Loading respiratory_support with all columns and 2247 hospitalization filters...
✅ All required tables loaded for SOFA computation


In [56]:
# Convert medication units to mcg/kg/min for SOFA computation
print("Converting medication units to mcg/kg/min for SOFA...")

# Define preferred units for SOFA medications
preferred_units = {
    'norepinephrine': 'mcg/kg/min',
    'epinephrine': 'mcg/kg/min',
    'dopamine': 'mcg/kg/min',
    'dobutamine': 'mcg/kg/min'
}

print(f"Converting {len(preferred_units)} medications: {list(preferred_units.keys())}")

# Convert units (uses vitals table for weight data)
co.convert_dose_units_for_continuous_meds(
    preferred_units=preferred_units,
    override = True, 
    save_to_table=True  # Saves to co.medication_admin_continuous.df_converted
)

# Check conversion results
conversion_counts = co.medication_admin_continuous.conversion_counts

print("\n=== Conversion Summary ===")
print(f"Total conversion records: {len(conversion_counts):,}")

# Check for conversion failures
success_count = conversion_counts[conversion_counts['_convert_status'] == 'success']['count'].sum()
total_count = conversion_counts['count'].sum()

print(f"Successful conversions: {success_count:,} / {total_count:,} ({100*success_count/total_count:.1f}%)")

# Show any failed conversions
failed_conversions = conversion_counts[conversion_counts['_convert_status'] != 'success']
if len(failed_conversions) > 0:
    print(f"\n⚠️ Found {len(failed_conversions)} conversion issues:")
    for _, row in failed_conversions.head(10).iterrows():
        print(f"  {row['med_category']}: {row['_clean_unit']} → {row['_convert_status']} ({row['count']} records)")
else:
    print("✅ All conversions successful!")

print("\n✅ Medication unit conversion completed")

Converting medication units to mcg/kg/min for SOFA...
Converting 4 medications: ['norepinephrine', 'epinephrine', 'dopamine', 'dobutamine']
No weight_kg column found, adding the most recent from vitals
Cannot accommodate the conversion to the following preferred units: {'meq/hr', 'meq/kg/hr', 'ppm'}. Consult the function documentation for a list of acceptable units.

=== Conversion Summary ===
Total conversion records: 89
Successful conversions: 1,497,656 / 1,530,705 (97.8%)

⚠️ Found 21 conversion issues:
  angiotensin: ng/kg/min → cannot convert to a weighted unit if weight_kg is missing (253 records)
  argatroban: mcg/kg/min → cannot convert to a weighted unit if weight_kg is missing (1 records)
  cangrelor: mcg/kg/min → cannot convert to a weighted unit if weight_kg is missing (8 records)
  cisatracurium: mcg/kg/min → cannot convert to a weighted unit if weight_kg is missing (42 records)
  dexmedetomidine: mcg/kg/hr → cannot convert to a weighted unit if weight_kg is missing (358 r

In [57]:
meds = co.medication_admin_continuous.df
unique_hosp_ids = meds['hospitalization_id'].nunique()
print(f"Unique hospitalization_id in meds df: {unique_hosp_ids}")

Unique hospitalization_id in meds df: 2225


In [58]:
sofa_cohort_df.dtypes()

TypeError: 'Series' object is not callable

In [ ]:
sofa_scores = co.compute_sofa_scores(
    cohort_df=sofa_cohort_df,
    id_name='encounter_block'
)

In [ ]:
sofa_scores

In [ ]:
# ============================================================================
# SOFA Score Summary Analysis
# ============================================================================
print("\n" + "=" * 80)
print("SOFA Score Summary for CRRT Cohort")
print("=" * 80)

# Assuming your SOFA dataframe is called 'sofa_df' or similar
# Replace 'sofa_df' with your actual dataframe name
sofa_df = sofa_scores.copy()
print(f"\n📊 Dataset Overview:")
print(f"   Total encounters: {len(sofa_df):,}")
print(f"   Total columns: {sofa_df.shape[1]}")

# ============================================================================
# Completeness Analysis
# ============================================================================
print("\n" + "-" * 80)
print("Data Completeness")
print("-" * 80)

sofa_columns = ['sofa_cv_97', 'sofa_coag', 'sofa_liver', 'sofa_resp', 
                'sofa_cns', 'sofa_renal', 'sofa_total']

print(f"\n{'Component':<20} {'Available':<15} {'Missing':<15} {'% Complete':<12}")
print("-" * 62)

for col in sofa_columns:
    available = sofa_df[col].notna().sum()
    missing = sofa_df[col].isna().sum()
    pct_complete = (available / len(sofa_df)) * 100
    print(f"{col:<20} {available:>6,} ({pct_complete:>5.1f}%)   {missing:>6,} ({100-pct_complete:>5.1f}%)")

# ============================================================================
# SOFA Component Score Distributions
# ============================================================================
print("\n" + "-" * 80)
print("SOFA Component Score Distributions")
print("-" * 80)

for col in sofa_columns:
    valid_scores = sofa_df[col].dropna()
    
    if len(valid_scores) > 0:
        print(f"\n{col.upper()}:")
        print(f"   N = {len(valid_scores):,}")
        print(f"   Mean ± SD: {valid_scores.mean():.2f} ± {valid_scores.std():.2f}")
        print(f"   Median [IQR]: {valid_scores.median():.1f} [{valid_scores.quantile(0.25):.1f}-{valid_scores.quantile(0.75):.1f}]")
        print(f"   Range: {valid_scores.min():.0f} - {valid_scores.max():.0f}")
        
        # Value counts for component scores
        if col != 'sofa_total':  # Component scores are 0-4
            print(f"   Distribution:")
            value_counts = valid_scores.value_counts().sort_index()
            for score, count in value_counts.items():
                print(f"     Score {int(score)}: {count:>5,} ({count/len(valid_scores)*100:>5.1f}%)")

# ============================================================================
# Total SOFA Score Categories
# ============================================================================
print("\n" + "-" * 80)
print("Total SOFA Score Categories")
print("-" * 80)

valid_total = sofa_df['sofa_total'].dropna()

if len(valid_total) > 0:
    print(f"\nClinical Severity Categories (based on total SOFA):")
    print(f"   {'Category':<25} {'N':<10} {'%'}")
    print(f"   {'-'*45}")
    
    # Standard SOFA severity categories
    categories = [
        ("Low (0-6)", (valid_total <= 6).sum()),
        ("Moderate (7-9)", ((valid_total >= 7) & (valid_total <= 9)).sum()),
        ("High (10-12)", ((valid_total >= 10) & (valid_total <= 12)).sum()),
        ("Very High (13-14)", ((valid_total >= 13) & (valid_total <= 14)).sum()),
        ("Extreme (≥15)", (valid_total >= 15).sum())
    ]
    
    for category, count in categories:
        pct = (count / len(valid_total)) * 100
        print(f"   {category:<25} {count:<10,} {pct:>5.1f}%")

# ============================================================================
# P/F Ratio Analysis
# ============================================================================
print("\n" + "-" * 80)
print("P/F Ratio Analysis")
print("-" * 80)

pf_available = sofa_df['p_f'].notna().sum()
pf_imputed_available = sofa_df['p_f_imputed'].notna().sum()

print(f"\nP/F Ratio Availability:")
print(f"   Original P/F: {pf_available:,} ({pf_available/len(sofa_df)*100:.1f}%)")
print(f"   Imputed P/F: {pf_imputed_available:,} ({pf_imputed_available/len(sofa_df)*100:.1f}%)")
print(f"   Imputation helped: {pf_imputed_available - pf_available:,} additional encounters")

if pf_imputed_available > 0:
    pf_stats = sofa_df['p_f_imputed'].describe()
    print(f"\nP/F Ratio Statistics (using imputed):")
    print(f"   Mean ± SD: {pf_stats['mean']:.1f} ± {pf_stats['std']:.1f}")
    print(f"   Median [IQR]: {pf_stats['50%']:.1f} [{pf_stats['25%']:.1f}-{pf_stats['75%']:.1f}]")
    print(f"   Range: {pf_stats['min']:.1f} - {pf_stats['max']:.1f}")

# ============================================================================
# Missing Pattern Analysis
# ============================================================================
print("\n" + "-" * 80)
print("Missing Data Patterns")
print("-" * 80)

# Check how many encounters have complete SOFA scores
complete_components = sofa_df[['sofa_cv_97', 'sofa_coag', 'sofa_liver', 
                                'sofa_resp', 'sofa_cns', 'sofa_renal']].notna().all(axis=1).sum()

print(f"\nComplete SOFA Assessments:")
print(f"   All 6 components available: {complete_components:,} ({complete_components/len(sofa_df)*100:.1f}%)")

# Count missing components per encounter
missing_per_encounter = sofa_df[['sofa_cv_97', 'sofa_coag', 'sofa_liver', 
                                  'sofa_resp', 'sofa_cns', 'sofa_renal']].isna().sum(axis=1)

print(f"\n   Distribution of missing components:")
for n_missing in range(7):
    count = (missing_per_encounter == n_missing).sum()
    if count > 0:
        print(f"     {n_missing} components missing: {count:,} ({count/len(sofa_df)*100:.1f}%)")

# ============================================================================
# Correlation Matrix
# ============================================================================
print("\n" + "-" * 80)
print("SOFA Component Correlations")
print("-" * 80)

correlation_cols = ['sofa_cv_97', 'sofa_coag', 'sofa_liver', 
                    'sofa_resp', 'sofa_cns', 'sofa_renal']
corr_matrix = sofa_df[correlation_cols].corr()

print("\nCorrelation between SOFA components:")
print(corr_matrix.round(2))

print("\n" + "=" * 80)
print("✅ SOFA Score Summary Complete")
print("=" * 80)

In [ ]:
outcomes_with_sofa = outcomes_df.merge(sofa_scores, on='encounter_block', how='left')
outcomes_with_sofa